<a href="https://colab.research.google.com/github/MestDash/PID/blob/main/notebooks/NN_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the packages




In [ ]:
import os
import math
import random
import statistics
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, RobustScaler, QuantileTransformer, PowerTransformer, LabelEncoder
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_curve
from sklearn.utils.class_weight import compute_class_weight
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, RandomOverSampler

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, regularizers, losses, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, LeakyReLU
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TerminateOnNaN
from tensorflow.keras.utils import to_categorical, plot_model, Sequence
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy

# Some functions

In [ ]:
def stratified_split(df: pd.DataFrame, test_size: float = 0.2, random_state: int = 17):
    X = df.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
    y = df['y']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=test_size,
        stratify=y,
        random_state=random_state
    )

    return X_train, X_test, y_train, y_test


def labelize(df):
    df = df.copy()

    def assign_label(row):
        # Handle both dash and no-dash variants
        if row['IUIS'] in ['No arguments for lymphoid-PID', 'No arguments for lymphoid PID']:
            return 'DC'
        else:
            main_label = row['IUIS'].split(':')[0].strip()
            if main_label == 'III':
                try:
                    first_digit = int(str(row['IUIS extended'])[0])
                    if first_digit == 4:
                        return 'III.4'
                    else:
                        return 'III.1-3'
                except (ValueError, IndexError):
                    return 'III.1-3'  # default if parsing fails
            else:
                return main_label

    df['y'] = df.apply(assign_label, axis=1)
    return df


RANDOM_STATE = 42
N_SPLITS = 5
CORR_THRESHOLD = 0.95
N_EPOCHS_BIN = 50
N_EPOCHS_MULTI = 50
BATCH_SIZE = 32
SMOTE_IMBALANCE_THRESHOLD = 0.5

np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

# Target classes (canonical order)
ALL_CLASSES = ["DC", "I", "II", "III.1-3", "III.4", "IV"]
STAGE3_CLASSES = ["II", "III.1-3", "III.4", "IV"]  # in this order for softmax head

class FeatureFilter(BaseEstimator, TransformerMixin):
    def __init__(self, corr_threshold=0.95):
        self.corr_threshold = corr_threshold
        self.to_drop_ = []

    def fit(self, X, y=None):
        # Keep only numeric columns
        numeric_df = X.select_dtypes(include=[np.number])

        # Drop constant features
        constant_features = [col for col in numeric_df.columns if numeric_df[col].nunique() <= 1]

        # Drop highly correlated features
        corr_matrix = numeric_df.corr().abs()
        upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        high_corr = [column for column in upper_triangle.columns if any(upper_triangle[column] > self.corr_threshold)]

        # Merge and store
        self.to_drop_ = list(set(constant_features + high_corr))
        print('These features will be dropped:')
        print(self.to_drop_)
        return self

    def transform(self, X):
        return X.drop(columns=self.to_drop_, errors='ignore')


def build_preprocessor_binary(X_train: pd.DataFrame):
    X_train = X_train.copy()
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

    cat_pipeline = Pipeline(
        steps=[
            ("impute", SimpleImputer(strategy="constant", fill_value="NA")),
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
        ]
    )

    numeric_pipe = Pipeline(steps=[
        ("scale",     QuantileTransformer(n_quantiles=X_train.shape[0])),
        ("impute", KNNImputer(n_neighbors=5)),
        ("minmax",      MinMaxScaler())
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", cat_pipeline, cat_cols)
        ],
        remainder="drop"
    )
    pre.fit(X_train)
    return pre


def build_preprocessor_multi(X_train: pd.DataFrame):
    X_train = X_train.copy()
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

    cat_pipeline = Pipeline(
        steps=[
            ("impute", SimpleImputer(strategy="constant", fill_value="NA")),
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
        ]
    )

    numeric_pipe = Pipeline(steps=[
        ("scale",     PowerTransformer()),
        ("impute", KNNImputer(n_neighbors=5))
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", cat_pipeline, cat_cols)
        ],
        remainder="drop"
    )
    pre.fit(X_train)
    return pre


# Custom transformer for conditional log1p
class ConditionalLog1p(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.cols_to_transform = None

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        # store which columns need log1p (max > 1)
        self.cols_to_transform = [
            col for col in X.columns if X[col].max() > 1
        ]
        return self

    def transform(self, X):
        X = pd.DataFrame(X).copy()
        for col in self.cols_to_transform:
            X[col] = np.log1p(X[col])
        return X.values

def build_stage1_labels(y):
    # I vs Rest
    return (y == "I").astype(int)

def build_stage2_labels(y):
    # DC vs Rest (but we train Stage2 on the subset y != I)
    mask = (y != "I")
    y2 = (y[mask] == "DC").astype(int)
    return mask, y2

def build_stage3_labels(y):
    # Multiclass on {II, IIIa, IIIb, IV}; we train Stage3 on y not in {I, DC}
    mask = ~y.isin(["I", "DC"])
    y3 = y[mask]
    # Map to indices in STAGE3_CLASSES order
    y3_idx = y3.map({c: i for i, c in enumerate(STAGE3_CLASSES)})
    return mask, y3_idx

def soft_gated_combine_probs(pI, pDC, pStage3_rows, thr_I=0.5, thr_DC=0.5, sharpness=10.0):
    """
    Softly combine probabilities from Stage 1, Stage 2, and Stage 3.

    pI:    (n,) probability of "I"
    pDC:   (n,) probability of "DC"
    pStage3_rows: (n,4) probabilities over [II, IIIa, IIIb, IV]
    thr_I: threshold for "I" gating
    thr_DC: threshold for "DC" gating
    sharpness: steepness of the gating sigmoid
    Returns:
      final_probs: (n, 6) in class order ALL_CLASSES
    """
    def soft_gate(prob, thr, sharpness):
        return 1 / (1 + np.exp(-sharpness * (prob - thr)))

    n = len(pI)
    final = np.zeros((n, 6), dtype=float)
    idx = {c: i for i, c in enumerate(ALL_CLASSES)}

    for i in range(n):
        # --- Soft-threshold scaling with smooth gates ---
        g_I  = soft_gate(pI[i], thr_I, sharpness)
        g_DC = soft_gate(pDC[i], thr_DC, sharpness)

        # Allocate probabilities
        p_I = g_I
        p_DC = (1 - g_I) * g_DC
        rest = (1 - g_I) * (1 - g_DC) * pStage3_rows[i]

        final[i, idx["I"]] = p_I
        final[i, idx["DC"]] = p_DC
        for j, cls in enumerate(STAGE3_CLASSES):
            final[i, idx[cls]] = rest[j]

        # Normalize
        s = final[i].sum()
        if s > 0:
            final[i] /= s
        else:
            final[i, :] = 1.0 / len(ALL_CLASSES)

    return final


def maybe_smote(X_tr, y_tr, is_multiclass=False):
    cnt = Counter(y_tr)
    if len(cnt) <= 1:
        return X_tr, y_tr  # nothing to do
    minc, maxc = min(cnt.values()), max(cnt.values())
    if (minc / maxc) < SMOTE_IMBALANCE_THRESHOLD:
        if is_multiclass:
            sm = SMOTE(random_state=RANDOM_STATE)
        else:
            sm = SMOTE(random_state=RANDOM_STATE)
        X_tr, y_tr = sm.fit_resample(X_tr, y_tr)
        print("SMOTE oversampling:", Counter(y_tr))
    return X_tr, y_tr

def plot_confusion_matrix_relative(cm, class_names, title="Confusion Matrix (Row-normalized)"):
    # Normalize per row (class)
    cm_normalized = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
    cm_normalized = np.nan_to_num(cm_normalized)  # replace NaN from divide-by-zero

    plt.figure(figsize=(6, 5))
    ax = sns.heatmap(
        cm_normalized,
        annot=cm,
        fmt="d",
        cmap="Blues",
        xticklabels=class_names,
        yticklabels=class_names,
        cbar=True,
        vmin=0.0, vmax=1.0,  # relative per row
        linecolor="black",   # border color
        linewidths=0.5       # thin border width
    )

    # Add outer border to match gridlines
    ax.add_patch(patches.Rectangle((0, 0), cm.shape[0], cm.shape[1],
                                   fill=False, edgecolor='black', lw=0.5, clip_on=False))

    plt.title(title)
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()

def find_optimal_threshold(y_true, y_prob, metric="balanced_accuracy", num_thresholds=200):
    """
    Find the threshold that maximizes a given metric.

    Parameters
    ----------
    y_true : array-like
        True binary labels (0/1).
    y_prob : array-like
        Predicted probabilities for the positive class.
    metric : str
        Metric to optimize. One of {"balanced_accuracy", "f1", "precision", "recall"}.
    num_thresholds : int
        Number of thresholds to evaluate between 0 and 1.

    Returns
    -------
    best_thresh : float
        Threshold that maximizes the chosen metric.
    best_score : float
        Score achieved at the best threshold.
    """
    # Choose metric function
    metric_funcs = {
        "balanced_accuracy": balanced_accuracy_score,
        "f1": f1_score,
        "precision": precision_score,
        "recall": recall_score,
    }
    if metric not in metric_funcs:
        raise ValueError(f"Unknown metric '{metric}'. Choose from {list(metric_funcs.keys())}")

    best_thresh, best_score = 0.5, -1
    thresholds = np.linspace(0.0, 1.0, num_thresholds)

    for t in thresholds:
        preds = (y_prob >= t).astype(int)
        try:
            score = metric_funcs[metric](y_true, preds)
        except ValueError:
            # Can happen if no positive predictions
            continue
        if score > best_score:
            best_thresh, best_score = t, score

    print(f"[Threshold Optimization] Best {metric}: {best_score:.4f} at threshold={best_thresh:.3f}")
    return best_thresh

def plot_history(history):
    # Plot curves
    plt.figure(figsize=(12, 6))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label="Train Acc")
    plt.plot(history.history['val_accuracy'], label="Val Acc")
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.ylim((0,1.1))

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label="Train Loss")
    plt.plot(history.history['val_loss'], label="Val Loss")
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    #plt.ylim((0,2))

    plt.show()

# Scaling benchmark

In [ ]:
# Standard NN for comparison
def build_binary(input_dim):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),

        layers.Dense(64, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-4)),
        #layers.BatchNormalization(),
        layers.Dropout(0.3),

        layers.Dense(32, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-4)),
        #layers.BatchNormalization(),
        layers.Dropout(0.3),

        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name="auc")]
    )
    return model

def build_multiclass(input_dim, num_classes):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),

        layers.Dense(128, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-4)),
        #layers.BatchNormalization(),
        layers.Dropout(0.3),

        layers.Dense(96, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-4)),
        #layers.BatchNormalization(),
        layers.Dropout(0.3),

        layers.Dense(64, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-4)),
        #layers.BatchNormalization(),
        layers.Dropout(0.3),

        layers.Dense(32, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-4)),
        #layers.BatchNormalization(),
        layers.Dropout(0.2),

        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=3,
    min_lr=1e-7
)

callbacks = [early_stop, reduce_lr]

In [ ]:
# Preprocessing pipeline
def build_preprocessor(X_train: pd.DataFrame):
    X_train = X_train.copy()
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

    cat_pipeline = Pipeline(
        steps=[
            ("impute", SimpleImputer(strategy="constant", fill_value="NA")),
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
        ]
    )

    numeric_pipe = Pipeline(steps=[
        ("scale",     StandardScaler()),
        ("impute", KNNImputer(n_neighbors=5)),
        ("minmax",      MinMaxScaler())
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", cat_pipeline, cat_cols)
        ],
        remainder="drop"
    )
    pre.fit(X_train)
    return pre

In [ ]:
def run_cascade_cv(X, y, n_splits: int = 5):

    stage1_acc, stage2_acc, stage3_acc = [], [], []
    fold_reports = []

    skf_outer = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    print(f"Starting {n_splits}-split CV (3 train / 1 val / 1 test) on {len(X)} samples...")

    for fold, (trainval_idx, test_idx) in enumerate(skf_outer.split(X, y), 1):
        # Split outer: test = 1 fold, trainval = 4 folds
        X_trainval, y_trainval = X.iloc[trainval_idx], y.iloc[trainval_idx]
        X_test_raw, y_test = X.iloc[test_idx], y.iloc[test_idx]

        # Inner split: 3 folds train, 1 fold val
        skf_inner = StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE)
        inner_train_idx, val_idx = next(skf_inner.split(X_trainval, y_trainval))
        X_train_raw, y_train = X_trainval.iloc[inner_train_idx], y_trainval.iloc[inner_train_idx]
        X_val_raw, y_val = X_trainval.iloc[val_idx], y_trainval.iloc[val_idx]

        print(f"\n==================== Fold {fold} ====================")
        print(f"Train: {len(X_train_raw)}, Val: {len(X_val_raw)}, Test: {len(X_test_raw)}")

        # ---- Feature filter + preprocess (fit only on train)
        ff = FeatureFilter(corr_threshold=CORR_THRESHOLD)
        ff.fit(X_train_raw)
        X_train_ff = ff.transform(X_train_raw)
        X_val_ff = ff.transform(X_val_raw)
        X_test_ff = ff.transform(X_test_raw)

        pre = build_preprocessor(X_train_ff)
        X_train_prep = pre.transform(X_train_ff)
        X_val_prep = pre.transform(X_val_ff)
        X_test_prep = pre.transform(X_test_ff)

        in_dim = X_train_prep.shape[1]

        # ========= Stage 1: I vs Rest =========
        print("\n-- Stage 1 (I vs Rest) --")
        y1_train = build_stage1_labels(y_train)
        y1_val   = build_stage1_labels(y_val)
        y1_test  = build_stage1_labels(y_test)
        stage1_labels = {0: "Rest", 1: "I"}

        cw1 = compute_class_weight(class_weight='balanced', classes=np.unique(y1_train), y=y1_train)
        cw1_dict = {int(cls): float(w) for cls, w in zip(np.unique(y1_train), cw1)}
        print("Class weights (Stage 1):", cw1_dict)

        m1 = build_binary(in_dim)
        m1.fit(X_train_prep, y1_train.values,
               validation_data=(X_val_prep, y1_val.values),
               epochs=50,
               batch_size=32,
               callbacks=callbacks,
               verbose=0,
               class_weight=cw1_dict)

        p1_val = m1.predict(X_val_prep, verbose=0).ravel()
        thresh1 = find_optimal_threshold(y1_val, p1_val)
        print(f"Optimal threshold (Stage 1): {thresh1:.3f}")

        p1_test = m1.predict(X_test_prep, verbose=0).ravel()
        pred1_test = (p1_test >= thresh1).astype(int)

        print(classification_report(y1_test, pred1_test, digits=4, zero_division=0))
        print("Balanced Accuracy:", balanced_accuracy_score(y1_test, pred1_test))
        cm1 = confusion_matrix(y1_test, pred1_test)
        #plot_confusion_matrix_relative(cm1, class_names=["Rest", "I"], title="Stage 1 (I vs Rest)")
        stage1_acc.append(balanced_accuracy_score(y1_test, pred1_test))

        # ========= Stage 2: DC vs Rest =========
        print("\n-- Stage 2 (DC vs Rest) --")
        mask2_train, y2_train = build_stage2_labels(y_train)
        mask2_val,   y2_val   = build_stage2_labels(y_val)
        mask2_test,  y2_test  = build_stage2_labels(y_test)

        X2_train = X_train_prep[mask2_train.values]
        X2_val   = X_val_prep[mask2_val.values]
        X2_test  = X_test_prep[mask2_test.values]

        cw2 = compute_class_weight(class_weight='balanced', classes=np.unique(y2_train), y=y2_train)
        cw2_dict = {int(cls): float(w) for cls, w in zip(np.unique(y2_train), cw2)}
        print("Class weights (Stage 2):", cw2_dict)

        m2 = build_binary(in_dim)
        m2.fit(X2_train, y2_train.values,
               validation_data=(X2_val, y2_val.values),
               epochs=50,
               batch_size=32,
               callbacks=callbacks,
               verbose=0,
               class_weight=cw2_dict)

        p2_val = m2.predict(X2_val, verbose=0).ravel()
        thresh2 = find_optimal_threshold(y2_val, p2_val)
        print(f"Optimal threshold (Stage 2): {thresh2:.3f}")

        p2_test = m2.predict(X2_test, verbose=0).ravel()
        pred2_test = (p2_test >= thresh2).astype(int)

        print(classification_report(y2_test, pred2_test, digits=4, zero_division=0))
        print("Balanced Accuracy:", balanced_accuracy_score(y2_test, pred2_test))
        cm2 = confusion_matrix(y2_test, pred2_test)
        #plot_confusion_matrix_relative(cm2, class_names=["PID", "DC"], title="Stage 2 (DC vs Rest)")
        stage2_acc.append(balanced_accuracy_score(y2_test, pred2_test))

        # ========= Stage 3: Multiclass =========
        print("\n-- Stage 3 (Multiclass) --")
        mask3_train, y3_train = build_stage3_labels(y_train)
        mask3_val,   y3_val   = build_stage3_labels(y_val)
        mask3_test,  y3_test  = build_stage3_labels(y_test)

        X3_train = X_train_prep[mask3_train.values]
        X3_val   = X_val_prep[mask3_val.values]
        X3_test  = X_test_prep[mask3_test.values]

        classes = np.unique(y3_train)
        cw3 = compute_class_weight(class_weight='balanced', classes=classes, y=y3_train)
        cw3_dict = {int(cls): float(w) for cls, w in zip(classes, cw3)}
        print("Class weights (Stage 3):", cw3_dict)

        m3 = build_multiclass(in_dim, len(STAGE3_CLASSES))
        m3.fit(X3_train, y3_train.values,
               validation_data=(X3_val, y3_val.values),
               epochs=50,
               batch_size=32,
               callbacks=callbacks,
               verbose=0,
               class_weight=cw3_dict)

        p3_test = m3.predict(X3_test, verbose=0)
        pred3_test = np.argmax(p3_test, axis=1)

        print(classification_report(y3_test, pred3_test, digits=4, zero_division=0))
        print("Balanced Accuracy:", balanced_accuracy_score(y3_test, pred3_test))
        cm3 = confusion_matrix(y3_test, pred3_test)
        #plot_confusion_matrix_relative(cm3, class_names=STAGE3_CLASSES, title="Stage 3 (Multiclass)")
        stage3_acc.append(balanced_accuracy_score(y3_test, pred3_test))

        # ========= Overall predictions (Soft-Gated) =========
        pI_all  = m1.predict(X_test_prep, verbose=0).ravel()
        pDC_all = m2.predict(X_test_prep, verbose=0).ravel()
        pS3_all = m3.predict(X_test_prep, verbose=0)

        final_probs = soft_gated_combine_probs(pI_all, pDC_all, pS3_all)
        final_idx = np.argmax(final_probs, axis=1)
        final_preds = [ALL_CLASSES[i] for i in final_idx]

        print("\n== Soft-Gated Overall ==")
        print(classification_report(y_test, final_preds, labels=ALL_CLASSES, digits=4, zero_division=0))
        print("Balanced Accuracy:", balanced_accuracy_score(y_test, final_preds))
        cm_final = confusion_matrix(y_test, final_preds, labels=ALL_CLASSES)
        plot_confusion_matrix_relative(cm_final, class_names=ALL_CLASSES, title="Final Soft-Gated Confusion Matrix")

        fold_reports.append({
            "fold": fold,
            "y_true": y_test.reset_index(drop=True),
            "y_pred": pd.Series(final_preds, index=y_test.index).reset_index(drop=True)
        })

    # ===== Summary =====
    y_true_all = pd.concat([fr["y_true"] for fr in fold_reports], axis=0)
    y_pred_all = pd.concat([fr["y_pred"] for fr in fold_reports], axis=0)
    print("\n==================== CV SUMMARY ====================")
    print(classification_report(y_true_all, y_pred_all, labels=ALL_CLASSES, digits=4, zero_division=0))
    print("Balanced Accuracy:", balanced_accuracy_score(y_true_all, y_pred_all))
    cm_summary = confusion_matrix(y_true_all, y_pred_all, labels=ALL_CLASSES)
    plot_confusion_matrix_relative(cm_summary, class_names=ALL_CLASSES, title="CV Summary Confusion Matrix")

    print("\n==================== Accuracies ====================")
    print([float(v) for v in stage1_acc])
    print([float(v) for v in stage2_acc])
    print([float(v) for v in stage3_acc])
    print("Mean balanced accuracy:", [
        np.mean(stage1_acc), np.mean(stage2_acc), np.mean(stage3_acc)
    ])

    return fold_reports, [np.mean(stage1_acc), np.mean(stage2_acc), np.mean(stage3_acc)]

In [ ]:
df = pd.read_csv('df_cluster.csv')
df = labelize(df)
X_train = df.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = df['y']

CORR_THRESHOLD = 0.9
'''
def run_multiple_cv(X, y, n_splits: int = N_SPLITS, n_repeats: int = 5):
    all_reports = []
    all_b_acc = []

    for repeat in range(1, n_repeats + 1):
        print(f"\n\n==================== REPEAT {repeat}/{n_repeats} ====================")

        fold_reports, b_acc_means = run_cascade_cv(X, y, n_splits=n_splits)
        all_reports.extend(fold_reports)
        all_b_acc.append(b_acc_means)

    print("\n==================== FINAL SUMMARY ====================")
    print("All balanced accuracies:", all_b_acc)
    #print("Mean balanced accuracy:", statistics.mean(all_b_acc))
    #se = np.std(all_b_acc, ddof=1) / np.sqrt(len(all_b_acc))
    #print("Standard Error:", se)

    return all_reports, all_b_acc
'''
all_reports, all_b_acc = run_multiple_flat_cv(X_train, y_train, n_splits=5, n_repeats=5)

In [ ]:
df = pd.read_csv('df_cluster.csv')
df = labelize(df)
X_train = df.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = df['y']

# Preprocessing pipeline
def build_preprocessor(X_train: pd.DataFrame):
    X_train = X_train.copy()
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

    cat_pipeline = Pipeline(
        steps=[
            ("impute", SimpleImputer(strategy="constant", fill_value="NA")),
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
        ]
    )

    numeric_pipe = Pipeline(steps=[
        ("scale",     RobustScaler()),
        ("impute", KNNImputer(n_neighbors=5)),
        ("minmax",      MinMaxScaler())
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", cat_pipeline, cat_cols)
        ],
        remainder="drop"
    )
    pre.fit(X_train)
    return pre

all_reports, all_b_acc = run_multiple_flat_cv(X_train, y_train, n_splits=5, n_repeats=5)

In [ ]:
df = pd.read_csv('df_cluster.csv')
df = labelize(df)
X_train = df.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = df['y']

# Preprocessing pipeline
def build_preprocessor(X_train: pd.DataFrame):
    X_train = X_train.copy()
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

    cat_pipeline = Pipeline(
        steps=[
            ("impute", SimpleImputer(strategy="constant", fill_value="NA")),
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
        ]
    )

    numeric_pipe = Pipeline(steps=[
        ("scale",     QuantileTransformer()),
        ("impute", KNNImputer(n_neighbors=5)),
        ("minmax",      MinMaxScaler())
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", cat_pipeline, cat_cols)
        ],
        remainder="drop"
    )
    pre.fit(X_train)
    return pre

all_reports, all_b_acc = run_multiple_flat_cv(X_train, y_train, n_splits=5, n_repeats=5)

In [ ]:
df = pd.read_csv('df_cluster.csv')
df = labelize(df)
X_train = df.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = df['y']

# Preprocessing pipeline
def build_preprocessor(X_train: pd.DataFrame):
    X_train = X_train.copy()
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

    cat_pipeline = Pipeline(
        steps=[
            ("impute", SimpleImputer(strategy="constant", fill_value="NA")),
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
        ]
    )

    numeric_pipe = Pipeline(steps=[
        ("scale",     PowerTransformer()),
        ("impute", KNNImputer(n_neighbors=5)),
        ("minmax",      MinMaxScaler())
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", cat_pipeline, cat_cols)
        ],
        remainder="drop"
    )
    pre.fit(X_train)
    return pre

all_reports, all_b_acc = run_multiple_flat_cv(X_train, y_train, n_splits=5, n_repeats=5)

In [ ]:
df = pd.read_csv('df_cluster.csv')
df = labelize(df)
X_train = df.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = df['y']

# Preprocessing pipeline
def build_preprocessor(X_train: pd.DataFrame):
    X_train = X_train.copy()
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

    cat_pipeline = Pipeline(
        steps=[
            ("impute", SimpleImputer(strategy="constant", fill_value="NA")),
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
        ]
    )

    numeric_pipe = Pipeline(steps=[
        ("scale",     ConditionalLog1p()),
        ("impute", KNNImputer(n_neighbors=5)),
        ("minmax",      MinMaxScaler())
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", cat_pipeline, cat_cols)
        ],
        remainder="drop"
    )
    pre.fit(X_train)
    return pre

all_reports, all_b_acc = run_multiple_flat_cv(X_train, y_train, n_splits=5, n_repeats=5)

In [ ]:
def build_flat_nn(input_dim, num_classes):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(256, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),

        layers.Dense(128, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),

        layers.Dense(64, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),

        layers.Dense(32, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.BatchNormalization(),
        layers.Dropout(0.2),

        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


callbacks = [
    EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True, verbose=0),
    ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=3, min_lr=1e-7, verbose=0),
]

def run_flat_cv(X, y, n_splits: int = 5):  # force 5 splits (3 train, 1 val, 1 test)

    # Encode labels
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    class_names = le.classes_

    skf_outer = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    fold_reports = []

    print(f"Starting {n_splits}-fold CV (3-train / 1-val / 1-test) on {len(X)} samples...")

    for fold, (trainval_idx, test_idx) in enumerate(skf_outer.split(X, y_encoded), 1):
        print(f"\n==================== Outer Fold {fold} ====================")

        # Split into outer train+val vs test
        X_trainval, X_test = X.iloc[trainval_idx].copy(), X.iloc[test_idx].copy()
        y_trainval, y_test = y_encoded[trainval_idx], y_encoded[test_idx]

        # Now split trainval into inner-train and inner-val (3 folds vs 1 fold)
        skf_inner = StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE)
        inner_train_idx, inner_val_idx = next(skf_inner.split(X_trainval, y_trainval))

        X_train, X_val = X_trainval.iloc[inner_train_idx], X_trainval.iloc[inner_val_idx]
        y_train, y_val = y_trainval[inner_train_idx], y_trainval[inner_val_idx]

        # ---- Feature filter
        ff = FeatureFilter(corr_threshold=CORR_THRESHOLD)
        ff.fit(X_train)
        X_train_ff = ff.transform(X_train)
        X_val_ff = ff.transform(X_val)
        X_test_ff = ff.transform(X_test)

        pre = build_preprocessor(X_train_ff)
        X_train_prep = pre.transform(X_train_ff)
        X_val_prep = pre.transform(X_val_ff)
        X_test_prep = pre.transform(X_test_ff)

        input_dim = X_train_prep.shape[1]

        # ---- Class weights
        cw = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
        cw_dict = {cls: w for cls, w in zip(np.unique(y_train), cw)}

        # ---- Build and train model
        model = build_flat_nn(input_dim, num_classes=6)
        model.fit(X_train_prep, y_train,
                  validation_data=(X_val_prep, y_val),
                  epochs=50,
                  batch_size=32,
                  callbacks=callbacks,
                  verbose=0,
                  class_weight=cw_dict
                  )

        # ---- Predictions on outer test fold
        y_pred_prob = model.predict(X_test_prep, verbose=0)
        y_pred_idx = np.argmax(y_pred_prob, axis=1)

        # Decode to original string labels
        y_test_labels = le.inverse_transform(y_test)
        y_pred_labels = le.inverse_transform(y_pred_idx)

        print(classification_report(y_test_labels, y_pred_labels, labels=class_names, digits=4, zero_division=0))
        print("Balanced Accuracy:", balanced_accuracy_score(y_test_labels, y_pred_labels))
        cm = confusion_matrix(y_test_labels, y_pred_labels, labels=class_names)
        print("Confusion Matrix:\n", cm)
        #plot_confusion_matrix_relative(cm, class_names=class_names, title=f"Outer Fold {fold} Confusion Matrix")

        fold_reports.append({
            "fold": fold,
            "y_true": pd.Series(y_test_labels).reset_index(drop=True),
            "y_pred": pd.Series(y_pred_labels).reset_index(drop=True)
        })

    # ===== CV summary =====
    y_true_all = pd.concat([fr["y_true"] for fr in fold_reports], axis=0)
    y_pred_all = pd.concat([fr["y_pred"] for fr in fold_reports], axis=0)

    print("\n==================== CV SUMMARY ====================")
    print(classification_report(y_true_all, y_pred_all, labels=class_names, digits=4, zero_division=0))
    print("Balanced Accuracy:", balanced_accuracy_score(y_true_all, y_pred_all))
    cm_summary = confusion_matrix(y_true_all, y_pred_all, labels=class_names)
    print("Confusion Matrix:\n", cm_summary)
    #plot_confusion_matrix_relative(cm_summary, class_names=class_names, title="CV Summary Confusion Matrix")
    overall_bACC = balanced_accuracy_score(y_true_all, y_pred_all)
    return fold_reports, overall_bACC

In [ ]:
df = pd.read_csv('df_cluster.csv')
df = labelize(df)
X_train = df.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = df['y']

CORR_THRESHOLD = 0.9

def run_multiple_flat_cv(X, y, n_splits: int = N_SPLITS, n_repeats: int = 5):
    all_reports = []
    all_b_acc = []

    for repeat in range(1, n_repeats + 1):
        print(f"\n\n==================== REPEAT {repeat}/{n_repeats} ====================")

        fold_reports, b_acc = run_flat_cv(X, y, n_splits=n_splits)
        all_reports.extend(fold_reports)
        all_b_acc.append(b_acc)

    print("\n==================== FINAL SUMMARY ====================")
    print("All balanced accuracies:", all_b_acc)
    #print("Mean balanced accuracy:", statistics.mean(all_b_acc))
    #se = np.std(all_b_acc, ddof=1) / np.sqrt(len(all_b_acc))
    #print("Standard Error:", se)

    return all_reports, all_b_acc

all_reports, all_b_acc = run_multiple_flat_cv(X_train, y_train, n_splits=5, n_repeats=5)

# Optimized

In [ ]:
# Preprocessing pipeline
def build_preprocessor(X_train: pd.DataFrame, stage):
    X_train = X_train.copy()
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

    cat_pipeline = Pipeline(
        steps=[
            ("impute", SimpleImputer(strategy="constant", fill_value="NA")),
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
        ]
    )

    if stage == 1:
        numeric_pipe = Pipeline(steps=[
            ("scale",     QuantileTransformer(n_quantiles=X_train.shape[0])),
            ("impute", KNNImputer(n_neighbors=5)),
            ("minmax",      MinMaxScaler())
        ])

    elif stage == 2:
        numeric_pipe = Pipeline(steps=[
            ("scale",     ConditionalLog1p()),
            ("impute", KNNImputer(n_neighbors=5)),
            ("minmax",      MinMaxScaler())
        ])

    elif stage == 3:
        numeric_pipe = Pipeline(steps=[
            ("scale",     PowerTransformer()),
            ("impute", KNNImputer(n_neighbors=5))
        ])

    elif stage == 4:
        numeric_pipe = Pipeline(steps=[
            ("scale",     RobustScaler()),
            ("impute", KNNImputer(n_neighbors=5)),
            ("minmax",      MinMaxScaler())
        ])

    pre = ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", cat_pipeline, cat_cols)
        ],
        remainder="drop"
    )
    pre.fit(X_train)
    return pre

In [ ]:
def create_model(input_dim, stage, params, is_binary):
    units = params["units"]
    learning_rate = params["learning_rate"]
    dropout_rate = params["dropout_rate"]
    num_hidden_layers = params["num_hidden_layers"]
    l2_reg = params["l2_reg"]
    activation = params["activation"]
    optimizer_name = params["optimizer"]
    model = Sequential()
    model.add(Input(shape=(input_dim,)))

    # Hidden layers
    for _ in range(num_hidden_layers - 1):
        model.add(Dense(units, activation=activation, kernel_regularizer=l2(l2_reg)))
        if not stage == 1 or stage == 3:
            model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))

    if stage == 1 or stage == 2:
        model.add(Dense(64, activation=activation, kernel_regularizer=l2(l2_reg)))
        if stage == 1:
            model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))
        model.add(Dense(32, activation=activation, kernel_regularizer=l2(l2_reg)))
        if stage == 1:
            model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))
    else:
        model.add(Dense(128, activation=activation, kernel_regularizer=l2(l2_reg)))
        if stage == 3:
            model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))
        model.add(Dense(64, activation=activation, kernel_regularizer=l2(l2_reg)))
        if stage == 3:
            model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))
        model.add(Dense(32, activation=activation, kernel_regularizer=l2(l2_reg)))
        if stage == 3:
            model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))

    # Output layer
    if is_binary:
        model.add(Dense(1, activation="sigmoid", kernel_regularizer=l2(l2_reg)))
        loss_fn = "binary_crossentropy"
    else:
        if stage == 3:
            model.add(Dense(4, activation="softmax", kernel_regularizer=l2(l2_reg)))
            loss_fn = "sparse_categorical_crossentropy"
        elif stage == 4:
            model.add(Dense(6, activation="softmax", kernel_regularizer=l2(l2_reg)))
            loss_fn = "sparse_categorical_crossentropy"

    # Select optimizer
    if optimizer_name == "adam":
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == "sgd":
        optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
    elif optimizer_name == "rmsprop":
        optimizer = RMSprop(learning_rate=learning_rate)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

    model.compile(
        optimizer=optimizer,
        loss=loss_fn,
        metrics=["accuracy"],
    )

    #filename = f"stage_{stage}.h5"
    #model.save(filename)

    return model

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=3,
    min_lr=1e-7
)

callbacks = [early_stop, reduce_lr]

In [ ]:
def balanced_accuracy_from_cm(cm: np.ndarray) -> float:
    """
    Compute balanced accuracy directly from a confusion matrix.
    Works for both binary and multiclass cases.
    """
    with np.errstate(divide="ignore", invalid="ignore"):
        recalls = np.diag(cm) / cm.sum(axis=1)
        recalls = recalls[~np.isnan(recalls)]  # drop NaNs if a class has no samples
    return np.mean(recalls)


def run_cascade_cv(X, y, n_splits: int = 5):

    stage1_acc, stage2_acc, stage3_acc = [], [], []
    fold_reports = []

    # ===== Initialize accumulators =====
    cm1_total = np.zeros((2, 2), dtype=int)  # Stage 1: binary Rest vs I
    cm2_total = np.zeros((2, 2), dtype=int)  # Stage 2: binary PID vs DC
    cm3_total = np.zeros((len(STAGE3_CLASSES), len(STAGE3_CLASSES)), dtype=int)  # Stage 3: multiclass
    cm_final_total = np.zeros((len(ALL_CLASSES), len(ALL_CLASSES)), dtype=int)   # Soft-gated final

    skf_outer = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    print(f"Starting {n_splits}-split CV (3 train / 1 val / 1 test) on {len(X)} samples...")

    for fold, (trainval_idx, test_idx) in enumerate(skf_outer.split(X, y), 1):
        # Split outer: test = 1 fold, trainval = 4 folds
        X_trainval, y_trainval = X.iloc[trainval_idx], y.iloc[trainval_idx]
        X_test_raw, y_test = X.iloc[test_idx], y.iloc[test_idx]

        # Inner split: 3 folds train, 1 fold val
        skf_inner = StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE)
        inner_train_idx, val_idx = next(skf_inner.split(X_trainval, y_trainval))
        X_train_raw, y_train = X_trainval.iloc[inner_train_idx], y_trainval.iloc[inner_train_idx]
        X_val_raw, y_val = X_trainval.iloc[val_idx], y_trainval.iloc[val_idx]

        print(f"\n==================== Fold {fold} ====================")
        print(f"Train: {len(X_train_raw)}, Val: {len(X_val_raw)}, Test: {len(X_test_raw)}")

        # ---- Feature filter + preprocess (fit only on train)
        ff = FeatureFilter(corr_threshold=CORR_THRESHOLD)
        ff.fit(X_train_raw)
        X_train_ff = ff.transform(X_train_raw)
        X_val_ff = ff.transform(X_val_raw)
        X_test_ff = ff.transform(X_test_raw)

        # ========= Stage 1: I vs Rest =========
        stage = 1
        pre1 = build_preprocessor(X_train_ff, stage)
        X1_train_prep = pre1.transform(X_train_ff)
        X1_val_prep = pre1.transform(X_val_ff)
        X1_test_prep = pre1.transform(X_test_ff)
        in_dim = X1_train_prep.shape[1]

        print("\n-- Stage 1 (I vs Rest) --")
        y1_train = build_stage1_labels(y_train)
        y1_val   = build_stage1_labels(y_val)
        y1_test  = build_stage1_labels(y_test)

        cw1 = compute_class_weight(class_weight='balanced', classes=np.unique(y1_train), y=y1_train)
        cw1_dict = {int(cls): float(w) for cls, w in zip(np.unique(y1_train), cw1)}

        m1 = create_model(in_dim, stage, params1, is_binary=True)
        m1.fit(X1_train_prep, y1_train.values,
               validation_data=(X1_val_prep, y1_val.values),
               epochs=params1["epochs"],
               batch_size=params1["batch_size"],
               callbacks=callbacks,
               verbose=0,
               class_weight=cw1_dict)

        p1_val = m1.predict(X1_val_prep, verbose=0).ravel()
        thresh1 = find_optimal_threshold(y1_val, p1_val)
        #if thresh1 >= 0.5:
        #    thresh1 = 0.4

        p1_test = m1.predict(X1_test_prep, verbose=0).ravel()
        pred1_test = (p1_test >= thresh1).astype(int)

        print(classification_report(y1_test, pred1_test, digits=4, zero_division=0))
        print("Balanced Accuracy:", balanced_accuracy_score(y1_test, pred1_test))
        cm1 = confusion_matrix(y1_test, pred1_test, labels=[0, 1])
        plot_confusion_matrix_relative(cm1, class_names=["Rest", "I"], title="Stage 1 (I vs Rest)")
        cm1_total += cm1
        stage1_acc.append(balanced_accuracy_score(y1_test, pred1_test))

        # ========= Stage 2: DC vs Rest =========
        stage = 2
        pre2 = build_preprocessor(X_train_ff, stage)
        X2_train_prep = pre2.transform(X_train_ff)
        X2_val_prep = pre2.transform(X_val_ff)
        X2_test_prep = pre2.transform(X_test_ff)

        print("\n-- Stage 2 (DC vs Rest) --")
        mask2_train, y2_train = build_stage2_labels(y_train)
        mask2_val,   y2_val   = build_stage2_labels(y_val)
        mask2_test,  y2_test  = build_stage2_labels(y_test)

        X2_train = X2_train_prep[mask2_train.values]
        X2_val   = X2_val_prep[mask2_val.values]
        X2_test  = X2_test_prep[mask2_test.values]

        cw2 = compute_class_weight(class_weight='balanced', classes=np.unique(y2_train), y=y2_train)
        cw2_dict = {int(cls): float(w) for cls, w in zip(np.unique(y2_train), cw2)}

        m2 = create_model(in_dim, stage, params2, is_binary=True)
        m2.fit(X2_train, y2_train.values,
               validation_data=(X2_val, y2_val.values),
               epochs=params2["epochs"],
               batch_size=params2["batch_size"],
               callbacks=callbacks,
               verbose=0,
               class_weight=cw2_dict)

        p2_val = m2.predict(X2_val, verbose=0).ravel()
        thresh2 = find_optimal_threshold(y2_val, p2_val)

        p2_test = m2.predict(X2_test, verbose=0).ravel()
        pred2_test = (p2_test >= thresh2).astype(int)

        print(classification_report(y2_test, pred2_test, digits=4, zero_division=0))
        print("Balanced Accuracy:", balanced_accuracy_score(y2_test, pred2_test))
        cm2 = confusion_matrix(y2_test, pred2_test, labels=[0, 1])
        plot_confusion_matrix_relative(cm2, class_names=["PID", "DC"], title="Stage 2 (DC vs Rest)")
        cm2_total += cm2
        stage2_acc.append(balanced_accuracy_score(y2_test, pred2_test))

        # ========= Stage 3: Multiclass =========
        stage = 3
        pre3 = build_preprocessor(X_train_ff, stage)
        X3_train_prep = pre3.transform(X_train_ff)
        X3_val_prep = pre3.transform(X_val_ff)
        X3_test_prep = pre3.transform(X_test_ff)

        print("\n-- Stage 3 (Multiclass) --")
        mask3_train, y3_train = build_stage3_labels(y_train)
        mask3_val,   y3_val   = build_stage3_labels(y_val)
        mask3_test,  y3_test  = build_stage3_labels(y_test)

        X3_train = X3_train_prep[mask3_train.values]
        X3_val   = X3_val_prep[mask3_val.values]
        X3_test  = X3_test_prep[mask3_test.values]

        classes = np.unique(y3_train)

        #ros = RandomOverSampler(random_state=17)
        #X3_train_res, y3_train_res = ros.fit_resample(X3_train, y3_train)

        cw3 = compute_class_weight(class_weight='balanced', classes=classes, y=y3_train)
        cw3_dict = {int(cls): float(w) for cls, w in zip(classes, cw3)}

        m3 = create_model(in_dim, stage, params3, is_binary=False)
        m3.fit(X3_train, y3_train.values,
               validation_data=(X3_val, y3_val.values),
               epochs=params3["epochs"],
               batch_size=params3["batch_size"],
               callbacks=callbacks,
               verbose=0,
               class_weight=cw3_dict)

        p3_test = m3.predict(X3_test, verbose=0)
        pred3_test = np.argmax(p3_test, axis=1)

        print(classification_report(y3_test, pred3_test, digits=4, zero_division=0))
        print("Balanced Accuracy:", balanced_accuracy_score(y3_test, pred3_test))
        cm3 = confusion_matrix(y3_test, pred3_test, labels=range(len(STAGE3_CLASSES)))
        plot_confusion_matrix_relative(cm3, class_names=STAGE3_CLASSES, title="Stage 3 (Multiclass)")
        cm3_total += cm3
        stage3_acc.append(balanced_accuracy_score(y3_test, pred3_test))

        # ========= Overall predictions (Soft-Gated) =========
        pI_all  = m1.predict(X1_test_prep, verbose=0).ravel()
        pDC_all = m2.predict(X2_test_prep, verbose=0).ravel()
        pS3_all = m3.predict(X3_test_prep, verbose=0)

        final_probs = soft_gated_combine_probs(pI_all, pDC_all, pS3_all, thresh1, thresh2)
        final_idx = np.argmax(final_probs, axis=1)
        final_preds = [ALL_CLASSES[i] for i in final_idx]

        print("\n== Soft-Gated Overall ==")
        print(classification_report(y_test, final_preds, labels=ALL_CLASSES, digits=4, zero_division=0))
        print("Balanced Accuracy:", balanced_accuracy_score(y_test, final_preds))
        cm_final = confusion_matrix(y_test, final_preds, labels=ALL_CLASSES)
        plot_confusion_matrix_relative(cm_final, class_names=ALL_CLASSES, title="Final Soft-Gated Confusion Matrix")
        cm_final_total += cm_final

        fold_reports.append({
            "fold": fold,
            "y_true": y_test.reset_index(drop=True),
            "y_pred": pd.Series(final_preds, index=y_test.index).reset_index(drop=True)
        })

    # ===== After CV: show aggregated confusion matrices =====
    print("\n==================== AGGREGATED CONFUSION MATRICES ====================")
    plot_confusion_matrix_relative(cm1_total, class_names=["Rest", "I"], title="Stage 1 (Total)")
    plot_confusion_matrix_relative(cm2_total, class_names=["PID", "DC"], title="Stage 2 (Total)")
    plot_confusion_matrix_relative(cm3_total, class_names=STAGE3_CLASSES, title="Stage 3 (Total)")
    plot_confusion_matrix_relative(cm_final_total, class_names=ALL_CLASSES, title="Final Soft-Gated (Total)")

    # ===== Compute balanced accuracy from aggregated confusion matrices =====
    agg_stage1_acc = balanced_accuracy_from_cm(cm1_total)
    agg_stage2_acc = balanced_accuracy_from_cm(cm2_total)
    agg_stage3_acc = balanced_accuracy_from_cm(cm3_total)
    agg_final_acc  = balanced_accuracy_from_cm(cm_final_total)

    # ===== Summary =====
    y_true_all = pd.concat([fr["y_true"] for fr in fold_reports], axis=0)
    y_pred_all = pd.concat([fr["y_pred"] for fr in fold_reports], axis=0)
    print("\n==================== CV SUMMARY ====================")
    print(classification_report(y_true_all, y_pred_all, labels=ALL_CLASSES, digits=4, zero_division=0))
    print("Balanced Accuracy (Final Soft-Gated, aggregated):", agg_final_acc)

    print("\n==================== Accuracies ====================")
    print("Per-fold balanced accuracy:")
    print("Stage 1:", [float(v) for v in stage1_acc])
    print("Stage 2:", [float(v) for v in stage2_acc])
    print("Stage 3:", [float(v) for v in stage3_acc])
    print("Mean per-fold balanced accuracy:", [
        np.mean(stage1_acc), np.mean(stage2_acc), np.mean(stage3_acc)
    ])
    print("Aggregated balanced accuracy:")
    print("Stage 1:", agg_stage1_acc)
    print("Stage 2:", agg_stage2_acc)
    print("Stage 3:", agg_stage3_acc)
    print("Final Soft-Gated:", agg_final_acc)

    return fold_reports, {
        "mean_per_fold": [np.mean(stage1_acc), np.mean(stage2_acc), np.mean(stage3_acc)],
        "aggregated": [agg_stage1_acc, agg_stage2_acc, agg_stage3_acc, agg_final_acc]
    }

In [ ]:
def run_multiple_cvs(X, y, n_runs: int = 5, n_splits: int = 5):
    all_results = []

    for run in range(1, n_runs + 1):
        print(f"\n\n==================== CV RUN {run}/{n_runs} ====================")
        fold_reports, summary = run_cascade_cv(X, y, n_splits=n_splits)
        all_results.append(summary)

    # Summarize variance across runs
    mean_per_fold = np.array([res["mean_per_fold"] for res in all_results])
    aggregated = np.array([res["aggregated"] for res in all_results])

    print("\n\n==================== MULTI-RUN SUMMARY ====================")
    print("Mean per-fold balanced accuracy (across runs):")
    print("Stage 1: mean =", mean_per_fold[:,0].mean(), ", std =", mean_per_fold[:,0].std())
    print("Stage 2: mean =", mean_per_fold[:,1].mean(), ", std =", mean_per_fold[:,1].std())
    print("Stage 3: mean =", mean_per_fold[:,2].mean(), ", std =", mean_per_fold[:,2].std())

    print("\nAggregated balanced accuracy (across runs):")
    print("Stage 1: mean =", aggregated[:,0].mean(), ", std =", aggregated[:,0].std())
    print("Stage 2: mean =", aggregated[:,1].mean(), ", std =", aggregated[:,1].std())
    print("Stage 3: mean =", aggregated[:,2].mean(), ", std =", aggregated[:,2].std())
    print("Final Soft-Gated: mean =", aggregated[:,3].mean(), ", std =", aggregated[:,3].std())
    return all_results

params1 = {'units': 239, 'learning_rate': 0.0016, 'dropout_rate': 0.364, 'num_hidden_layers': 1, 'batch_size': 110, 'epochs': 46, 'l2_reg': 0.0039, 'activation': "selu", 'optimizer': "sgd"}
params2 = {'units': 167, 'learning_rate': 0.0054, 'dropout_rate': 0.3488, 'num_hidden_layers': 1, 'batch_size': 21, 'epochs': 50, 'l2_reg': 0.00612, 'activation': 'selu', 'optimizer': 'rmsprop'}
params3 = {'units': 86, 'learning_rate': 0.01, 'dropout_rate': 0.2, 'num_hidden_layers': 3, 'batch_size': 71, 'epochs': 86, 'l2_reg': 0.000001, 'activation': "selu", 'optimizer': "rmsprop"}
params4 = {'units': 150, 'learning_rate': 0.01, 'dropout_rate': 0.5, 'num_hidden_layers': 1, 'batch_size': 84, 'epochs': 88, 'l2_reg': 0.01, 'activation': "selu", 'optimizer': "sgd"}

train = pd.read_csv('df_cluster.csv')
test = pd.read_csv('df_cluster_VAL.csv')

train = labelize(train)
test = labelize(test)
test.columns = train.columns # some column names have slighlty different names

CORR_THRESHOLD = 0.9

X_train = train.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = train['y']
X_test = test.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_test = test['y']

run_multiple_cvs(X_train, y_train, n_runs=5, n_splits=3)

In [ ]:
def run_multi_cv(X, y, n_splits: int = 5):  # force 5 splits (3 train, 1 val, 1 test)

    # Encode labels
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    class_names = le.classes_

    skf_outer = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    fold_reports = []

    print(f"Starting {n_splits}-fold CV (3-train / 1-val / 1-test) on {len(X)} samples...")

    for fold, (trainval_idx, test_idx) in enumerate(skf_outer.split(X, y_encoded), 1):
        print(f"\n==================== Outer Fold {fold} ====================")

        # Split into outer train+val vs test
        X_trainval, X_test = X.iloc[trainval_idx].copy(), X.iloc[test_idx].copy()
        y_trainval, y_test = y_encoded[trainval_idx], y_encoded[test_idx]

        # Now split trainval into inner-train and inner-val (3 folds vs 1 fold)
        skf_inner = StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE)
        inner_train_idx, inner_val_idx = next(skf_inner.split(X_trainval, y_trainval))

        X_train, X_val = X_trainval.iloc[inner_train_idx], X_trainval.iloc[inner_val_idx]
        y_train, y_val = y_trainval[inner_train_idx], y_trainval[inner_val_idx]

        # ---- Feature filter
        ff = FeatureFilter(corr_threshold=CORR_THRESHOLD)
        ff.fit(X_train)
        X_train_ff = ff.transform(X_train)
        X_val_ff = ff.transform(X_val)
        X_test_ff = ff.transform(X_test)

        pre4 = build_preprocessor(X_train_ff, 4)
        X_train_prep = pre4.transform(X_train_ff)
        X_val_prep = pre4.transform(X_val_ff)
        X_test_prep = pre4.transform(X_test_ff)

        input_dim = X_train_prep.shape[1]

        # ---- Class weights
        cw = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
        cw_dict = {cls: w for cls, w in zip(np.unique(y_train), cw)}

        # ---- Build and train model
        model = create_model(input_dim, 4, params4, is_binary=False)
        model.fit(X_train_prep, y_train,
                  validation_data=(X_val_prep, y_val),
                  epochs=int(params4["epochs"]),
                  batch_size=int(params4["batch_size"]),
                  callbacks=callbacks,
                  verbose=0,
                  class_weight=cw_dict
                  )

        # ---- Predictions on outer test fold
        y_pred_prob = model.predict(X_test_prep, verbose=0)
        y_pred_idx = np.argmax(y_pred_prob, axis=1)

        # Decode to original string labels
        y_test_labels = le.inverse_transform(y_test)
        y_pred_labels = le.inverse_transform(y_pred_idx)

        print(classification_report(y_test_labels, y_pred_labels, labels=class_names, digits=4, zero_division=0))
        print("Balanced Accuracy:", balanced_accuracy_score(y_test_labels, y_pred_labels))
        cm = confusion_matrix(y_test_labels, y_pred_labels, labels=class_names)
        print("Confusion Matrix:\n", cm)
        plot_confusion_matrix_relative(cm, class_names=class_names, title=f"Outer Fold {fold} Confusion Matrix")

        fold_reports.append({
            "fold": fold,
            "y_true": pd.Series(y_test_labels).reset_index(drop=True),
            "y_pred": pd.Series(y_pred_labels).reset_index(drop=True)
        })

    # ===== CV summary =====
    y_true_all = pd.concat([fr["y_true"] for fr in fold_reports], axis=0)
    y_pred_all = pd.concat([fr["y_pred"] for fr in fold_reports], axis=0)

    print("\n==================== CV SUMMARY ====================")
    print(classification_report(y_true_all, y_pred_all, labels=class_names, digits=4, zero_division=0))
    print("Balanced Accuracy:", balanced_accuracy_score(y_true_all, y_pred_all))
    cm_summary = confusion_matrix(y_true_all, y_pred_all, labels=class_names)
    print("Confusion Matrix:\n", cm_summary)
    plot_confusion_matrix_relative(cm_summary, class_names=class_names, title="CV Summary Confusion Matrix")
    overall_bACC = balanced_accuracy_score(y_true_all, y_pred_all)
    return fold_reports, overall_bACC

In [ ]:
params4 = {'units': 150, 'learning_rate': 0.01, 'dropout_rate': 0.5, 'num_hidden_layers': 1, 'batch_size': 84, 'epochs': 88, 'l2_reg': 0.01, 'activation': "selu", 'optimizer': "sgd"}

df = pd.read_csv('df_cluster.csv')
df = labelize(df)
X_train = df.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = df['y']

CORR_THRESHOLD = 0.9

def run_multiple_cv(X, y, n_splits: int = N_SPLITS, n_repeats: int = 5):
    all_reports = []
    all_b_acc = []

    for repeat in range(1, n_repeats + 1):
        print(f"\n\n==================== REPEAT {repeat}/{n_repeats} ====================")

        fold_reports, b_acc = run_multi_cv(X, y, n_splits=n_splits)
        all_reports.extend(fold_reports)
        all_b_acc.append(b_acc)

    print("\n==================== FINAL SUMMARY ====================")
    print("All balanced accuracies:", all_b_acc)
    print("Mean balanced accuracy:", statistics.mean(all_b_acc))
    se = np.std(all_b_acc, ddof=1) / np.sqrt(len(all_b_acc))
    print("Standard Error:", se)

    return all_reports, all_b_acc

all_reports, all_b_acc = run_multiple_cv(X_train, y_train, n_splits=5, n_repeats=5)

# CV with Balanced Batching


In [ ]:
class BalancedBatchGenerator(Sequence):
    """
    Generates balanced batches from (X, y) for imbalanced classification problems.
    Each batch includes approximately equal samples from each class.
    """
    def __init__(self, X, y, batch_size=32):
        self.X = np.asarray(X)
        self.y = np.asarray(y)
        self.batch_size = batch_size
        self.classes, self.class_indices = np.unique(self.y, return_inverse=True)
        self.indices_per_class = {cls: np.where(self.y == cls)[0] for cls in self.classes}
        self.samples_per_class = max(1, batch_size // len(self.classes))

    def __len__(self):
        return int(np.ceil(len(self.y) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = []
        for cls in self.classes:
            cls_indices = np.random.choice(
                self.indices_per_class[cls],
                size=self.samples_per_class,
                replace=len(self.indices_per_class[cls]) < self.samples_per_class
            )
            batch_indices.extend(cls_indices)
        np.random.shuffle(batch_indices)
        X_batch = self.X[batch_indices]
        y_batch = self.y[batch_indices]
        return X_batch, y_batch


In [ ]:
def balanced_accuracy_from_cm(cm: np.ndarray) -> float:
    """
    Compute balanced accuracy directly from a confusion matrix.
    Works for both binary and multiclass cases.
    """
    with np.errstate(divide="ignore", invalid="ignore"):
        recalls = np.diag(cm) / cm.sum(axis=1)
        recalls = recalls[~np.isnan(recalls)]  # drop NaNs if a class has no samples
    return np.mean(recalls)


def run_cascade_cv(X, y, n_splits: int = 5):

    stage1_acc, stage2_acc, stage3_acc = [], [], []
    fold_reports = []

    # ===== Initialize accumulators =====
    cm1_total = np.zeros((2, 2), dtype=int)  # Stage 1: binary Rest vs I
    cm2_total = np.zeros((2, 2), dtype=int)  # Stage 2: binary PID vs DC
    cm3_total = np.zeros((len(STAGE3_CLASSES), len(STAGE3_CLASSES)), dtype=int)  # Stage 3: multiclass
    cm_final_total = np.zeros((len(ALL_CLASSES), len(ALL_CLASSES)), dtype=int)   # Soft-gated final

    skf_outer = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    print(f"Starting {n_splits}-split CV (3 train / 1 val / 1 test) on {len(X)} samples...")

    for fold, (trainval_idx, test_idx) in enumerate(skf_outer.split(X, y), 1):
        # Split outer: test = 1 fold, trainval = 4 folds
        X_trainval, y_trainval = X.iloc[trainval_idx], y.iloc[trainval_idx]
        X_test_raw, y_test = X.iloc[test_idx], y.iloc[test_idx]

        # Inner split: 3 folds train, 1 fold val
        skf_inner = StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE)
        inner_train_idx, val_idx = next(skf_inner.split(X_trainval, y_trainval))
        X_train_raw, y_train = X_trainval.iloc[inner_train_idx], y_trainval.iloc[inner_train_idx]
        X_val_raw, y_val = X_trainval.iloc[val_idx], y_trainval.iloc[val_idx]

        print(f"\n==================== Fold {fold} ====================")
        print(f"Train: {len(X_train_raw)}, Val: {len(X_val_raw)}, Test: {len(X_test_raw)}")

        # ---- Feature filter + preprocess (fit only on train)
        ff = FeatureFilter(corr_threshold=CORR_THRESHOLD)
        ff.fit(X_train_raw)
        X_train_ff = ff.transform(X_train_raw)
        X_val_ff = ff.transform(X_val_raw)
        X_test_ff = ff.transform(X_test_raw)

        # ========= Stage 1: I vs Rest =========
        stage = 1
        pre1 = build_preprocessor(X_train_ff, stage)
        X1_train_prep = pre1.transform(X_train_ff)
        X1_val_prep = pre1.transform(X_val_ff)
        X1_test_prep = pre1.transform(X_test_ff)
        in_dim = X1_train_prep.shape[1]

        print("\n-- Stage 1 (I vs Rest) --")
        y1_train = build_stage1_labels(y_train)
        y1_val   = build_stage1_labels(y_val)
        y1_test  = build_stage1_labels(y_test)

        #cw1 = compute_class_weight(class_weight='balanced', classes=np.unique(y1_train), y=y1_train)
        #cw1_dict = {int(cls): float(w) for cls, w in zip(np.unique(y1_train), cw1)}

        m1 = create_model(in_dim, stage, params1, is_binary=True)

        gen1 = BalancedBatchGenerator(X1_train_prep, y1_train.values, batch_size=params1["batch_size"])
        m1.fit(gen1,
               validation_data=(X1_val_prep, y1_val.values),
               epochs=params1["epochs"],
               callbacks=callbacks,
               verbose=0)
               #class_weight=cw1_dict)

        p1_val = m1.predict(X1_val_prep, verbose=0).ravel()
        thresh1 = find_optimal_threshold(y1_val, p1_val)
        #if thresh1 >= 0.5:
        #    thresh1 = 0.4

        p1_test = m1.predict(X1_test_prep, verbose=0).ravel()
        pred1_test = (p1_test >= thresh1).astype(int)

        print(classification_report(y1_test, pred1_test, digits=4, zero_division=0))
        print("Balanced Accuracy:", balanced_accuracy_score(y1_test, pred1_test))
        cm1 = confusion_matrix(y1_test, pred1_test, labels=[0, 1])
        plot_confusion_matrix_relative(cm1, class_names=["Rest", "I"], title="Stage 1 (I vs Rest)")
        cm1_total += cm1
        stage1_acc.append(balanced_accuracy_score(y1_test, pred1_test))

        # ========= Stage 2: DC vs Rest =========
        stage = 2
        pre2 = build_preprocessor(X_train_ff, stage)
        X2_train_prep = pre2.transform(X_train_ff)
        X2_val_prep = pre2.transform(X_val_ff)
        X2_test_prep = pre2.transform(X_test_ff)

        print("\n-- Stage 2 (DC vs Rest) --")
        mask2_train, y2_train = build_stage2_labels(y_train)
        mask2_val,   y2_val   = build_stage2_labels(y_val)
        mask2_test,  y2_test  = build_stage2_labels(y_test)

        X2_train = X2_train_prep[mask2_train.values]
        X2_val   = X2_val_prep[mask2_val.values]
        X2_test  = X2_test_prep[mask2_test.values]

        #cw2 = compute_class_weight(class_weight='balanced', classes=np.unique(y2_train), y=y2_train)
        #cw2_dict = {int(cls): float(w) for cls, w in zip(np.unique(y2_train), cw2)}

        m2 = create_model(in_dim, stage, params2, is_binary=True)

        gen2 = BalancedBatchGenerator(X2_train, y2_train.values, batch_size=params2["batch_size"])
        m2.fit(gen2,
               validation_data=(X2_val, y2_val.values),
               epochs=params2["epochs"],
               callbacks=callbacks,
               verbose=0)
               #class_weight=cw2_dict)


        p2_val = m2.predict(X2_val, verbose=0).ravel()
        thresh2 = find_optimal_threshold(y2_val, p2_val)

        p2_test = m2.predict(X2_test, verbose=0).ravel()
        pred2_test = (p2_test >= thresh2).astype(int)

        print(classification_report(y2_test, pred2_test, digits=4, zero_division=0))
        print("Balanced Accuracy:", balanced_accuracy_score(y2_test, pred2_test))
        cm2 = confusion_matrix(y2_test, pred2_test, labels=[0, 1])
        plot_confusion_matrix_relative(cm2, class_names=["PID", "DC"], title="Stage 2 (DC vs Rest)")
        cm2_total += cm2
        stage2_acc.append(balanced_accuracy_score(y2_test, pred2_test))

        # ========= Stage 3: Multiclass =========
        stage = 3
        pre3 = build_preprocessor(X_train_ff, stage)
        X3_train_prep = pre3.transform(X_train_ff)
        X3_val_prep = pre3.transform(X_val_ff)
        X3_test_prep = pre3.transform(X_test_ff)

        print("\n-- Stage 3 (Multiclass) --")
        mask3_train, y3_train = build_stage3_labels(y_train)
        mask3_val,   y3_val   = build_stage3_labels(y_val)
        mask3_test,  y3_test  = build_stage3_labels(y_test)

        X3_train = X3_train_prep[mask3_train.values]
        X3_val   = X3_val_prep[mask3_val.values]
        X3_test  = X3_test_prep[mask3_test.values]

        classes = np.unique(y3_train)

        #cw3 = compute_class_weight(class_weight='balanced', classes=classes, y=y3_train)
        #cw3_dict = {int(cls): float(w) for cls, w in zip(classes, cw3)}

        m3 = create_model(in_dim, stage, params3, is_binary=False)

        gen3 = BalancedBatchGenerator(X3_train, y3_train.values, batch_size=params3["batch_size"])
        m3.fit(gen3,
               validation_data=(X3_val, y3_val.values),
               epochs=params3["epochs"],
               callbacks=callbacks,
               verbose=0)
               #class_weight=cw3_dict)


        p3_test = m3.predict(X3_test, verbose=0)
        pred3_test = np.argmax(p3_test, axis=1)

        print(classification_report(y3_test, pred3_test, digits=4, zero_division=0))
        print("Balanced Accuracy:", balanced_accuracy_score(y3_test, pred3_test))
        cm3 = confusion_matrix(y3_test, pred3_test, labels=range(len(STAGE3_CLASSES)))
        plot_confusion_matrix_relative(cm3, class_names=STAGE3_CLASSES, title="Stage 3 (Multiclass)")
        cm3_total += cm3
        stage3_acc.append(balanced_accuracy_score(y3_test, pred3_test))

        # ========= Overall predictions (Soft-Gated) =========
        pI_all  = m1.predict(X1_test_prep, verbose=0).ravel()
        pDC_all = m2.predict(X2_test_prep, verbose=0).ravel()
        pS3_all = m3.predict(X3_test_prep, verbose=0)

        final_probs = soft_gated_combine_probs(pI_all, pDC_all, pS3_all, thresh1, thresh2)
        final_idx = np.argmax(final_probs, axis=1)
        final_preds = [ALL_CLASSES[i] for i in final_idx]

        print("\n== Soft-Gated Overall ==")
        print(classification_report(y_test, final_preds, labels=ALL_CLASSES, digits=4, zero_division=0))
        print("Balanced Accuracy:", balanced_accuracy_score(y_test, final_preds))
        cm_final = confusion_matrix(y_test, final_preds, labels=ALL_CLASSES)
        plot_confusion_matrix_relative(cm_final, class_names=ALL_CLASSES, title="Final Soft-Gated Confusion Matrix")
        cm_final_total += cm_final

        fold_reports.append({
            "fold": fold,
            "y_true": y_test.reset_index(drop=True),
            "y_pred": pd.Series(final_preds, index=y_test.index).reset_index(drop=True)
        })

    # ===== After CV: show aggregated confusion matrices =====
    print("\n==================== AGGREGATED CONFUSION MATRICES ====================")
    plot_confusion_matrix_relative(cm1_total, class_names=["Rest", "I"], title="Stage 1 (Total)")
    plot_confusion_matrix_relative(cm2_total, class_names=["PID", "DC"], title="Stage 2 (Total)")
    plot_confusion_matrix_relative(cm3_total, class_names=STAGE3_CLASSES, title="Stage 3 (Total)")
    plot_confusion_matrix_relative(cm_final_total, class_names=ALL_CLASSES, title="Final Soft-Gated (Total)")

    # ===== Compute balanced accuracy from aggregated confusion matrices =====
    agg_stage1_acc = balanced_accuracy_from_cm(cm1_total)
    agg_stage2_acc = balanced_accuracy_from_cm(cm2_total)
    agg_stage3_acc = balanced_accuracy_from_cm(cm3_total)
    agg_final_acc  = balanced_accuracy_from_cm(cm_final_total)

    # ===== Summary =====
    y_true_all = pd.concat([fr["y_true"] for fr in fold_reports], axis=0)
    y_pred_all = pd.concat([fr["y_pred"] for fr in fold_reports], axis=0)
    print("\n==================== CV SUMMARY ====================")
    print(classification_report(y_true_all, y_pred_all, labels=ALL_CLASSES, digits=4, zero_division=0))
    print("Balanced Accuracy (Final Soft-Gated, aggregated):", agg_final_acc)

    print("\n==================== Accuracies ====================")
    print("Per-fold balanced accuracy:")
    print("Stage 1:", [float(v) for v in stage1_acc])
    print("Stage 2:", [float(v) for v in stage2_acc])
    print("Stage 3:", [float(v) for v in stage3_acc])
    print("Mean per-fold balanced accuracy:", [
        np.mean(stage1_acc), np.mean(stage2_acc), np.mean(stage3_acc)
    ])
    print("Aggregated balanced accuracy:")
    print("Stage 1:", agg_stage1_acc)
    print("Stage 2:", agg_stage2_acc)
    print("Stage 3:", agg_stage3_acc)
    print("Final Soft-Gated:", agg_final_acc)

    return fold_reports, {
        "mean_per_fold": [np.mean(stage1_acc), np.mean(stage2_acc), np.mean(stage3_acc)],
        "aggregated": [agg_stage1_acc, agg_stage2_acc, agg_stage3_acc, agg_final_acc]
    }

In [ ]:
def run_multiple_cvs(X, y, n_runs: int = 5, n_splits: int = 5):
    all_results = []

    for run in range(1, n_runs + 1):
        print(f"\n\n==================== CV RUN {run}/{n_runs} ====================")
        fold_reports, summary = run_cascade_cv(X, y, n_splits=n_splits)
        all_results.append(summary)

    # Summarize variance across runs
    mean_per_fold = np.array([res["mean_per_fold"] for res in all_results])
    aggregated = np.array([res["aggregated"] for res in all_results])

    print("\n\n==================== MULTI-RUN SUMMARY ====================")
    print("Mean per-fold balanced accuracy (across runs):")
    print("Stage 1: mean =", mean_per_fold[:,0].mean(), ", std =", mean_per_fold[:,0].std())
    print("Stage 2: mean =", mean_per_fold[:,1].mean(), ", std =", mean_per_fold[:,1].std())
    print("Stage 3: mean =", mean_per_fold[:,2].mean(), ", std =", mean_per_fold[:,2].std())

    print("\nAggregated balanced accuracy (across runs):")
    print("Stage 1: mean =", aggregated[:,0].mean(), ", std =", aggregated[:,0].std())
    print("Stage 2: mean =", aggregated[:,1].mean(), ", std =", aggregated[:,1].std())
    print("Stage 3: mean =", aggregated[:,2].mean(), ", std =", aggregated[:,2].std())
    print("Final Soft-Gated: mean =", aggregated[:,3].mean(), ", std =", aggregated[:,3].std())
    return all_results

params1 = {'units': 239, 'learning_rate': 0.0016, 'dropout_rate': 0.364, 'num_hidden_layers': 1, 'batch_size': 110, 'epochs': 46, 'l2_reg': 0.0039, 'activation': "selu", 'optimizer': "sgd"}
params2 = {'units': 167, 'learning_rate': 0.0054, 'dropout_rate': 0.3488, 'num_hidden_layers': 1, 'batch_size': 21, 'epochs': 50, 'l2_reg': 0.00612, 'activation': 'selu', 'optimizer': 'rmsprop'}
params3 = {'units': 86, 'learning_rate': 0.01, 'dropout_rate': 0.2, 'num_hidden_layers': 3, 'batch_size': 71, 'epochs': 86, 'l2_reg': 0.000001, 'activation': "selu", 'optimizer': "rmsprop"}
params4 = {'units': 150, 'learning_rate': 0.01, 'dropout_rate': 0.5, 'num_hidden_layers': 1, 'batch_size': 84, 'epochs': 88, 'l2_reg': 0.01, 'activation': "selu", 'optimizer': "sgd"}

train = pd.read_csv('df_cluster.csv')
test = pd.read_csv('df_cluster_VAL.csv')

train = labelize(train)
test = labelize(test)
test.columns = train.columns # some column names have slighlty different names

CORR_THRESHOLD = 0.9

X_train = train.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = train['y']
X_test = test.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_test = test['y']

run_multiple_cvs(X_train, y_train, n_runs=5, n_splits=3)

# Testing on the holdout set

Before continuing, make sure the optimized preprocessing pipeline and the *create_model* function from the "Optimized" section are loaded.

In [ ]:
def run_cascade_TEST(X_train, y_train, X_holdout, y_holdout):

    def train_stage(X_tr, X_val, X_te, y_tr, y_val, y_te, stage_num, params, is_binary,
                   stage_name, class_names, run_name):

        cw = compute_class_weight(class_weight="balanced", classes=np.unique(y_val), y=y_val)
        cw_dict = {int(cls): float(w) for cls, w in zip(np.unique(y_val), cw)}

        model = create_model(X_tr.shape[1], stage_num, params, is_binary=is_binary)
        history = model.fit(X_tr, y_tr.values if hasattr(y_tr, 'values') else y_tr,
                          validation_data=(X_val, y_val.values),
                          epochs=params["epochs"],
                          batch_size=params["batch_size"],
                          callbacks=callbacks,
                          verbose=0,
                          class_weight=cw_dict)
        plot_history(history)

        if is_binary:
            p_val = model.predict(X_val, verbose=0).ravel()
            thresh = max(find_optimal_threshold(y_val, p_val), 0.2 if stage_num == 2 else 0)
            print(f"Optimal threshold ({stage_name}): {thresh:.3f}")

            p_te = model.predict(X_te, verbose=0).ravel()
            pred_te = (p_te >= thresh).astype(int)
        else:
            thresh = None
            p_te = model.predict(X_te, verbose=0)
            pred_te = np.argmax(p_te, axis=1)

        print(classification_report(y_te, pred_te, digits=4, zero_division=0))
        acc = balanced_accuracy_score(y_te, pred_te)
        print("Balanced Accuracy:", acc)
        cm = confusion_matrix(y_te, pred_te)
        plot_confusion_matrix_relative(cm, class_names=class_names,
                                      title=f"{stage_name} - {run_name}")

        return model, thresh, acc

    def train_and_evaluate_single_run(X_val_raw, X_test_raw, y_val, y_test, run_name):

        # Feature filter and preprocessing
        ff = FeatureFilter(corr_threshold=CORR_THRESHOLD)
        ff.fit(X_train)
        X_tr_ff, X_val_ff, X_te_ff = [ff.transform(X) for X in [X_train, X_val_raw, X_test_raw]]

        preprocessors = [build_preprocessor(X_tr_ff, i) for i in [1, 2, 3]]
        X_stages = {
            'tr': [p.transform(X_tr_ff) for p in preprocessors],
            'val': [p.transform(X_val_ff) for p in preprocessors],
            'te': [p.transform(X_te_ff) for p in preprocessors]
        }

        stage_accs = []
        stage_preds = {}

        # Stage 1: I vs Rest
        print("\n-- Stage 1 (I vs Rest) --")
        y1_tr, y1_val, y1_te = [build_stage1_labels(y) for y in [y_train, y_val, y_test]]
        m1, thresh1, acc1 = train_stage(
            X_stages['tr'][0], X_stages['val'][0], X_stages['te'][0],
            y1_tr, y1_val, y1_te, 1, params1, True,
            "Stage 1 (I vs Rest)", ["Rest", "I"], run_name
        )
        stage_accs.append(acc1)
        p1_te = m1.predict(X_stages['te'][0], verbose=0).ravel()
        stage_preds['stage1'] = {'y_true': y1_te, 'y_pred': (p1_te >= thresh1).astype(int)}

        # Stage 2: DC vs Rest
        print("\n-- Stage 2 (DC vs Rest) --")
        masks_y2 = [build_stage2_labels(y) for y in [y_train, y_val, y_test]]
        X2_masked = [X_stages[k][1][masks_y2[i][0].values] for i, k in enumerate(['tr', 'val', 'te'])]
        y2_masked = [masks_y2[i][1] for i in range(3)]

        m2, thresh2, acc2 = train_stage(
            X2_masked[0], X2_masked[1], X2_masked[2],
            y2_masked[0], y2_masked[1], y2_masked[2], 2, params2, True,
            "Stage 2 (DC vs Rest)", ["PID", "DC"], run_name
        )
        stage_accs.append(acc2)
        p2_te = m2.predict(X2_masked[2], verbose=0).ravel()
        stage_preds['stage2'] = {'y_true': y2_masked[2], 'y_pred': (p2_te >= thresh2).astype(int)}

        # Stage 3: Multiclass
        print("\n-- Stage 3 (Multiclass) --")
        masks_y3 = [build_stage3_labels(y) for y in [y_train, y_val, y_test]]
        X3_masked = [X_stages[k][2][masks_y3[i][0].values] for i, k in enumerate(['tr', 'val', 'te'])]
        y3_masked = [masks_y3[i][1] for i in range(3)]

        m3, _, acc3 = train_stage(
            X3_masked[0], X3_masked[1], X3_masked[2],
            y3_masked[0], y3_masked[1], y3_masked[2], 3, params3, False,
            "Stage 3 (Multiclass)", STAGE3_CLASSES, run_name
        )
        stage_accs.append(acc3)
        p3_te = m3.predict(X3_masked[2], verbose=0)
        stage_preds['stage3'] = {'y_true': y3_masked[2].values, 'y_pred': np.argmax(p3_te, axis=1)}

        # Overall predictions
        pI_all = m1.predict(X_stages['te'][0], verbose=0).ravel()
        pDC_all = m2.predict(X_stages['te'][1], verbose=0).ravel()
        pS3_all = m3.predict(X_stages['te'][2], verbose=0)

        final_probs = soft_gated_combine_probs(pI_all, pDC_all, pS3_all, thresh1, thresh2)
        final_preds = [ALL_CLASSES[i] for i in np.argmax(final_probs, axis=1)]

        print(f"\n== Soft-Gated Overall (Test Set) - {run_name} ==")
        print(classification_report(y_test, final_preds, labels=ALL_CLASSES,
                                   target_names=ALL_CLASSES, digits=4, zero_division=0))
        acc_final = balanced_accuracy_score(y_test, final_preds)
        print("Balanced Accuracy:", acc_final)
        cm_final = confusion_matrix(y_test, final_preds, labels=ALL_CLASSES)
        plot_confusion_matrix_relative(cm_final, class_names=ALL_CLASSES,
                                      title=f"Final Soft-Gated - {run_name}")
        stage_accs.append(acc_final)

        return final_preds, stage_accs, stage_preds

    # Split holdout set
    sks = StratifiedKFold(n_splits=2, shuffle=True, random_state=RANDOM_STATE)
    idx_A, idx_B = next(sks.split(X_holdout, y_holdout))
    X_A, X_B = X_holdout.iloc[idx_A].copy(), X_holdout.iloc[idx_B].copy()
    y_A, y_B = y_holdout.iloc[idx_A].copy(), y_holdout.iloc[idx_B].copy()

    # Storage for aggregated results
    all_predictions = {}
    all_y_true = {}
    all_stage_accuracies = []
    all_stage_predictions = {'stage1': [], 'stage2': [], 'stage3': []}

    # Run 1: A=validation, B=test
    print("\n" + "="*60)
    print("RUN 1: Using Split A for validation, Split B for testing")
    print("="*60)

    preds_run1, accs_run1, stage_preds_run1 = train_and_evaluate_single_run(X_A, X_B, y_A, y_B, "Run 1")
    for i, idx in enumerate(idx_B):
        all_predictions[idx] = preds_run1[i]
        all_y_true[idx] = y_B.iloc[i]
    all_stage_accuracies.append(accs_run1)
    for stage in ['stage1', 'stage2', 'stage3']:
        all_stage_predictions[stage].append(stage_preds_run1[stage])

    # Run 2: B=validation, A=test (SWAP)
    print("\n" + "="*60)
    print("RUN 2: Using Split B for validation, Split A for testing (SWAP)")
    print("="*60)

    preds_run2, accs_run2, stage_preds_run2 = train_and_evaluate_single_run(X_B, X_A, y_B, y_A, "Run 2")
    for i, idx in enumerate(idx_A):
        all_predictions[idx] = preds_run2[i]
        all_y_true[idx] = y_A.iloc[i]
    all_stage_accuracies.append(accs_run2)
    for stage in ['stage1', 'stage2', 'stage3']:
        all_stage_predictions[stage].append(stage_preds_run2[stage])

    # Aggregated results
    print("\n" + "="*60)
    print("AGGREGATED RESULTS ACROSS ENTIRE HOLDOUT SET")
    print("="*60)

    # Stage 1 aggregated results
    print("\n" + "-"*60)
    print("STAGE 1 (I vs Rest) - AGGREGATED")
    print("-"*60)
    y1_true_all = np.concatenate([all_stage_predictions['stage1'][0]['y_true'],
                                   all_stage_predictions['stage1'][1]['y_true']])
    y1_pred_all = np.concatenate([all_stage_predictions['stage1'][0]['y_pred'],
                                   all_stage_predictions['stage1'][1]['y_pred']])
    print(classification_report(y1_true_all, y1_pred_all, digits=4, zero_division=0))
    bal_acc_stage1 = balanced_accuracy_score(y1_true_all, y1_pred_all)
    print(f"Balanced Accuracy: {bal_acc_stage1:.4f}")
    cm_stage1_agg = confusion_matrix(y1_true_all, y1_pred_all)
    plot_confusion_matrix_relative(cm_stage1_agg, class_names=["Rest", "I"],
                                   title="Stage 1 (I vs Rest) - Aggregated")

    # Stage 2 aggregated results
    print("\n" + "-"*60)
    print("STAGE 2 (DC vs Rest) - AGGREGATED")
    print("-"*60)
    y2_true_all = np.concatenate([all_stage_predictions['stage2'][0]['y_true'],
                                   all_stage_predictions['stage2'][1]['y_true']])
    y2_pred_all = np.concatenate([all_stage_predictions['stage2'][0]['y_pred'],
                                   all_stage_predictions['stage2'][1]['y_pred']])
    print(classification_report(y2_true_all, y2_pred_all, digits=4, zero_division=0))
    bal_acc_stage2 = balanced_accuracy_score(y2_true_all, y2_pred_all)
    print(f"Balanced Accuracy: {bal_acc_stage2:.4f}")
    cm_stage2_agg = confusion_matrix(y2_true_all, y2_pred_all)
    plot_confusion_matrix_relative(cm_stage2_agg, class_names=["PID", "DC"],
                                   title="Stage 2 (DC vs Rest) - Aggregated")

    # Stage 3 aggregated results
    print("\n" + "-"*60)
    print("STAGE 3 (Multiclass) - AGGREGATED")
    print("-"*60)
    y3_true_all = np.concatenate([all_stage_predictions['stage3'][0]['y_true'],
                                   all_stage_predictions['stage3'][1]['y_true']])
    y3_pred_all = np.concatenate([all_stage_predictions['stage3'][0]['y_pred'],
                                   all_stage_predictions['stage3'][1]['y_pred']])
    print(classification_report(y3_true_all, y3_pred_all, digits=4, zero_division=0))
    bal_acc_stage3 = balanced_accuracy_score(y3_true_all, y3_pred_all)
    print(f"Balanced Accuracy: {bal_acc_stage3:.4f}")
    cm_stage3_agg = confusion_matrix(y3_true_all, y3_pred_all)
    plot_confusion_matrix_relative(cm_stage3_agg, class_names=STAGE3_CLASSES,
                                   title="Stage 3 (Multiclass) - Aggregated")

    # Final soft-gated aggregated results
    print("\n" + "-"*60)
    print("SOFT-GATED OVERALL - AGGREGATED")
    print("-"*60)

    sorted_indices = sorted(all_predictions.keys())
    final_preds_all = [all_predictions[i] for i in sorted_indices]
    y_true_all = [all_y_true[i] for i in sorted_indices]

    print("\n== Aggregated Classification Report ==")
    print(classification_report(y_true_all, final_preds_all, labels=ALL_CLASSES,
                               target_names=ALL_CLASSES, digits=4, zero_division=0))

    bal_acc_aggregated = balanced_accuracy_score(y_true_all, final_preds_all)
    print(f"\nAggregated Balanced Accuracy: {bal_acc_aggregated:.4f}")

    cm_aggregated = confusion_matrix(y_true_all, final_preds_all, labels=ALL_CLASSES)
    plot_confusion_matrix_relative(cm_aggregated, class_names=ALL_CLASSES,
                                  title="Aggregated Final Confusion Matrix (Entire Holdout)")

    avg_stage_accs = np.mean(all_stage_accuracies, axis=0)
    print(f"\n== Average Stage Balanced Accuracies ==")
    print(f"Stage 1 (I vs Rest): {avg_stage_accs[0]:.4f}")
    print(f"Stage 2 (DC vs Rest): {avg_stage_accs[1]:.4f}")
    print(f"Stage 3 (Multiclass): {avg_stage_accs[2]:.4f}")
    print(f"Final (Soft-Gated): {avg_stage_accs[3]:.4f}")

    return final_preds_all, [bal_acc_stage1, bal_acc_stage2, bal_acc_stage3, bal_acc_aggregated]

In [ ]:
params1 = {'units': 239, 'learning_rate': 0.0016, 'dropout_rate': 0.364, 'num_hidden_layers': 1, 'batch_size': 110, 'epochs': 46, 'l2_reg': 0.0039, 'activation': "selu", 'optimizer': "sgd"}
params2 = {'units': 167, 'learning_rate': 0.0054, 'dropout_rate': 0.3488, 'num_hidden_layers': 1, 'batch_size': 21, 'epochs': 50, 'l2_reg': 0.00612, 'activation': 'selu', 'optimizer': 'rmsprop'}
params3 = {'units': 86, 'learning_rate': 0.01, 'dropout_rate': 0.2, 'num_hidden_layers': 3, 'batch_size': 71, 'epochs': 86, 'l2_reg': 0.000001, 'activation': "selu", 'optimizer': "rmsprop"}
params4 = {'units': 150, 'learning_rate': 0.01, 'dropout_rate': 0.5, 'num_hidden_layers': 1, 'batch_size': 84, 'epochs': 88, 'l2_reg': 0.01, 'activation': "selu", 'optimizer': "sgd"}

train = pd.read_csv('df_cluster.csv')
test = pd.read_csv('df_cluster_VAL.csv')

train = labelize(train)
test = labelize(test)
test.columns = train.columns # some column names have slighlty different names

CORR_THRESHOLD = 0.9

X_train = train.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = train['y']
X_test = test.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_test = test['y']

accs = []
for i in range(20):
    report, b_acc = run_cascade_TEST(X_train, y_train, X_test, y_test)
    accs.append(b_acc)
print(accs)

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=3,
    min_lr=1e-7
)

callbacks = [early_stop, reduce_lr]

def run_multi_TEST(X_train, y_train, X_holdout, y_holdout, params):
    """
    Train multiclass model on training set, swap validation/test splits to predict entire holdout.
    Returns predictions for entire holdout set and aggregated metrics.
    """

    def train_and_evaluate_single_run(X_val, X_test, y_val, y_test, run_name):
        """Helper function to train and evaluate one run"""

        # Encode labels
        le = LabelEncoder()
        y_train_enc, y_val_enc, y_test_enc = [le.fit_transform(y_train) if i == 0 else le.transform(y)
                                               for i, y in enumerate([y_train, y_val, y_test])]
        class_names = le.classes_

        # Feature filter and preprocessing
        ff = FeatureFilter(corr_threshold=CORR_THRESHOLD)
        ff.fit(X_train)
        X_tr_ff, X_val_ff, X_te_ff = [ff.transform(X) for X in [X_train, X_val, X_test]]

        pre_multi = build_preprocessor_multi(X_tr_ff)
        X_tr_prep, X_val_prep, X_te_prep = [pre_multi.transform(X) for X in [X_tr_ff, X_val_ff, X_te_ff]]
        input_dim = X_tr_prep.shape[1]

        # Class weights
        cw = compute_class_weight(class_weight="balanced", classes=np.unique(y_train_enc), y=y_train_enc)
        cw_dict = {cls: w for cls, w in zip(np.unique(y_train_enc), cw)}

        # Build and train model
        model = create_model(input_dim, 4, params, is_binary=False)
        history = model.fit(
            X_tr_prep, y_train_enc,
            validation_data=(X_val_prep, y_val_enc),
            epochs=int(params["epochs"]),
            batch_size=int(params["batch_size"]),
            callbacks=callbacks,
            verbose=0,
            class_weight=cw_dict
        )

        # Predictions
        y_pred_prob = model.predict(X_te_prep, verbose=1)
        y_pred_idx = np.argmax(y_pred_prob, axis=1)
        y_test_labels = le.inverse_transform(y_test_enc)
        y_pred_labels = le.inverse_transform(y_pred_idx)

        # Reports
        print(f"\n== Test Set Evaluation - {run_name} ==")
        print(classification_report(y_test_labels, y_pred_labels, labels=class_names, digits=4, zero_division=0))
        bal_acc = balanced_accuracy_score(y_test_labels, y_pred_labels)
        print("Balanced Accuracy:", bal_acc)
        cm = confusion_matrix(y_test_labels, y_pred_labels, labels=class_names)
        print("Confusion Matrix:\n", cm)
        plot_confusion_matrix_relative(cm, class_names=class_names, title=f"Test Confusion Matrix - {run_name}")

        return y_test_labels, y_pred_labels, bal_acc, class_names, history

    # Split holdout set
    sks = StratifiedKFold(n_splits=2, shuffle=True, random_state=RANDOM_STATE)
    idx_A, idx_B = next(sks.split(X_holdout, y_holdout))
    X_A, X_B = X_holdout.iloc[idx_A].copy(), X_holdout.iloc[idx_B].copy()
    y_A, y_B = y_holdout.iloc[idx_A].copy(), y_holdout.iloc[idx_B].copy()

    # Storage for aggregated results
    all_predictions = {}
    all_y_true = {}
    all_balanced_accs = []

    # Run 1: A=validation, B=test
    print("\n" + "="*60)
    print("RUN 1: Using Split A for validation, Split B for testing")
    print("="*60)

    y_test_1, y_pred_1, bal_acc_1, class_names, history = train_and_evaluate_single_run(X_A, X_B, y_A, y_B, "Run 1")
    for i, idx in enumerate(idx_B):
        all_predictions[idx] = y_pred_1[i]
        all_y_true[idx] = y_test_1[i]
    all_balanced_accs.append(bal_acc_1)

    # Run 2: B=validation, A=test (SWAP)
    print("\n" + "="*60)
    print("RUN 2: Using Split B for validation, Split A for testing (SWAP)")
    print("="*60)

    y_test_2, y_pred_2, bal_acc_2, _, history = train_and_evaluate_single_run(X_B, X_A, y_B, y_A, "Run 2")
    for i, idx in enumerate(idx_A):
        all_predictions[idx] = y_pred_2[i]
        all_y_true[idx] = y_test_2[i]
    all_balanced_accs.append(bal_acc_2)

    # Aggregated results
    print("\n" + "="*60)
    print("AGGREGATED RESULTS ACROSS ENTIRE HOLDOUT SET")
    print("="*60)

    sorted_indices = sorted(all_predictions.keys())
    final_preds_all = [all_predictions[i] for i in sorted_indices]
    y_true_all = [all_y_true[i] for i in sorted_indices]

    print("\n== Aggregated Classification Report ==")
    print(classification_report(y_true_all, final_preds_all, labels=class_names, digits=4, zero_division=0))

    bal_acc_aggregated = balanced_accuracy_score(y_true_all, final_preds_all)
    print(f"\nAggregated Balanced Accuracy: {bal_acc_aggregated:.4f}")

    cm_aggregated = confusion_matrix(y_true_all, final_preds_all, labels=class_names)
    print("Aggregated Confusion Matrix:\n", cm_aggregated)
    plot_confusion_matrix_relative(cm_aggregated, class_names=class_names,
                                   title="Aggregated Confusion Matrix (Entire Holdout)")

    avg_bal_acc = np.mean(all_balanced_accs)
    print(f"\n== Average Balanced Accuracy Across Runs ==")
    print(f"Run 1: {all_balanced_accs[0]:.4f}")
    print(f"Run 2: {all_balanced_accs[1]:.4f}")
    print(f"Average: {avg_bal_acc:.4f}")

    return (
        {"y_true": pd.Series(y_true_all).reset_index(drop=True),
         "y_pred": pd.Series(final_preds_all).reset_index(drop=True)},
        bal_acc_aggregated,
        history
    )

In [ ]:
params4 = {'units': 150, 'learning_rate': 0.01, 'dropout_rate': 0.5, 'num_hidden_layers': 1, 'batch_size': 84, 'epochs': 88, 'l2_reg': 0.01, 'activation': "selu", 'optimizer': "sgd"}

train = pd.read_csv('df_cluster.csv')
test = pd.read_csv('df_cluster_VAL.csv')
train = labelize(train)
test = labelize(test)
test.columns = train.columns # some column names have slighlty different names

X_train = train.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = train['y']
X_holdout = test.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_holdout =  test['y']

accs = []
for i in range(10):
    report, b_acc, history = run_multi_TEST(X_train, y_train, X_holdout, y_holdout, params4)
    accs.append(b_acc)
accs = [float(a) for a in accs]
print(accs)
print(statistics.mean(accs), statistics.pstdev(accs) / math.sqrt(len(accs)))

# Testing with balanced batching

Before continuing, make sure the optimized preprocessing pipeline and the *create_model* function from the "Optimized" section are loaded.

In [ ]:
class BalancedBatchGenerator(Sequence):
    """
    Generates balanced batches from (X, y) for imbalanced classification problems.
    Each batch includes approximately equal samples from each class.
    """
    def __init__(self, X, y, batch_size=32, **kwargs):
        super().__init__(**kwargs)
        self.X = np.asarray(X)
        self.y = np.asarray(y)
        self.batch_size = batch_size
        self.classes, self.class_indices = np.unique(self.y, return_inverse=True)
        self.indices_per_class = {cls: np.where(self.y == cls)[0] for cls in self.classes}
        self.samples_per_class = max(1, batch_size // len(self.classes))

    def __len__(self):
        return int(np.ceil(len(self.y) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = []
        for cls in self.classes:
            cls_indices = np.random.choice(
                self.indices_per_class[cls],
                size=self.samples_per_class,
                replace=len(self.indices_per_class[cls]) < self.samples_per_class
            )
            batch_indices.extend(cls_indices)
        np.random.shuffle(batch_indices)
        X_batch = self.X[batch_indices]
        y_batch = self.y[batch_indices]
        return X_batch, y_batch

In [ ]:
def run_cascade_TEST(X_train, y_train, X_holdout, y_holdout):
    """
    Train cascade on training set, swap validation/test splits to predict entire holdout.
    Returns predictions for entire holdout set and aggregated metrics.
    """

    def train_stage(X_tr, X_val, X_te, y_tr, y_val, y_te, stage_num, params, is_binary,
                   stage_name, class_names, run_name):
        """Helper to train and evaluate one stage"""
        gen = BalancedBatchGenerator(X_tr, y_tr.values if hasattr(y_tr, 'values') else y_tr,
                                     batch_size=params["batch_size"])

        model = create_model(X_tr.shape[1], stage_num, params, is_binary=is_binary)
        history = model.fit(gen,
                          validation_data=(X_val, y_val.values),
                          epochs=params["epochs"],
                          callbacks=callbacks,
                          verbose=0)
        plot_history(history)

        if is_binary:
            p_val = model.predict(X_val, verbose=0).ravel()
            thresh = max(find_optimal_threshold(y_val, p_val), 0.2 if stage_num == 2 else 0)
            print(f"Optimal threshold ({stage_name}): {thresh:.3f}")

            p_te = model.predict(X_te, verbose=0).ravel()
            pred_te = (p_te >= thresh).astype(int)
        else:
            thresh = None
            p_te = model.predict(X_te, verbose=0)
            pred_te = np.argmax(p_te, axis=1)

        print(classification_report(y_te, pred_te, digits=4, zero_division=0))
        acc = balanced_accuracy_score(y_te, pred_te)
        print("Balanced Accuracy:", acc)
        cm = confusion_matrix(y_te, pred_te)
        plot_confusion_matrix_relative(cm, class_names=class_names,
                                      title=f"{stage_name} - {run_name}")

        return model, thresh, acc

    def train_and_evaluate_single_run(X_val_raw, X_test_raw, y_val, y_test, run_name):
        """Helper function to train and evaluate one cascade run"""

        # Feature filter and preprocessing
        ff = FeatureFilter(corr_threshold=CORR_THRESHOLD)
        ff.fit(X_train)
        X_tr_ff, X_val_ff, X_te_ff = [ff.transform(X) for X in [X_train, X_val_raw, X_test_raw]]

        preprocessors = [build_preprocessor(X_tr_ff, i) for i in [1, 2, 3]]
        X_stages = {
            'tr': [p.transform(X_tr_ff) for p in preprocessors],
            'val': [p.transform(X_val_ff) for p in preprocessors],
            'te': [p.transform(X_te_ff) for p in preprocessors]
        }

        stage_accs = []

        # Stage 1: I vs Rest
        print("\n-- Stage 1 (I vs Rest) --")
        y1_tr, y1_val, y1_te = [build_stage1_labels(y) for y in [y_train, y_val, y_test]]
        m1, thresh1, acc1 = train_stage(
            X_stages['tr'][0], X_stages['val'][0], X_stages['te'][0],
            y1_tr, y1_val, y1_te, 1, params1, True,
            "Stage 1 (I vs Rest)", ["Rest", "I"], run_name
        )
        stage_accs.append(acc1)

        # Stage 2: DC vs Rest
        print("\n-- Stage 2 (DC vs Rest) --")
        masks_y2 = [build_stage2_labels(y) for y in [y_train, y_val, y_test]]
        X2_masked = [X_stages[k][1][masks_y2[i][0].values] for i, k in enumerate(['tr', 'val', 'te'])]
        y2_masked = [masks_y2[i][1] for i in range(3)]

        m2, thresh2, acc2 = train_stage(
            X2_masked[0], X2_masked[1], X2_masked[2],
            y2_masked[0], y2_masked[1], y2_masked[2], 2, params2, True,
            "Stage 2 (DC vs Rest)", ["PID", "DC"], run_name
        )
        stage_accs.append(acc2)

        # Stage 3: Multiclass
        print("\n-- Stage 3 (Multiclass) --")
        masks_y3 = [build_stage3_labels(y) for y in [y_train, y_val, y_test]]
        X3_masked = [X_stages[k][2][masks_y3[i][0].values] for i, k in enumerate(['tr', 'val', 'te'])]
        y3_masked = [masks_y3[i][1] for i in range(3)]

        m3, _, acc3 = train_stage(
            X3_masked[0], X3_masked[1], X3_masked[2],
            y3_masked[0], y3_masked[1], y3_masked[2], 3, params3, False,
            "Stage 3 (Multiclass)", STAGE3_CLASSES, run_name
        )
        stage_accs.append(acc3)

        # Overall predictions
        pI_all = m1.predict(X_stages['te'][0], verbose=0).ravel()
        pDC_all = m2.predict(X_stages['te'][1], verbose=0).ravel()
        pS3_all = m3.predict(X_stages['te'][2], verbose=0)

        final_probs = soft_gated_combine_probs(pI_all, pDC_all, pS3_all, thresh1, thresh2)
        final_preds = [ALL_CLASSES[i] for i in np.argmax(final_probs, axis=1)]

        print(f"\n== Soft-Gated Overall (Test Set) - {run_name} ==")
        print(classification_report(y_test, final_preds, labels=ALL_CLASSES,
                                   target_names=ALL_CLASSES, digits=4, zero_division=0))
        acc_final = balanced_accuracy_score(y_test, final_preds)
        print("Balanced Accuracy:", acc_final)
        cm_final = confusion_matrix(y_test, final_preds, labels=ALL_CLASSES)
        plot_confusion_matrix_relative(cm_final, class_names=ALL_CLASSES,
                                      title=f"Final Soft-Gated - {run_name}")
        stage_accs.append(acc_final)

        return final_preds, stage_accs

    # Split holdout set
    sks = StratifiedKFold(n_splits=2, shuffle=True, random_state=RANDOM_STATE)
    idx_A, idx_B = next(sks.split(X_holdout, y_holdout))
    X_A, X_B = X_holdout.iloc[idx_A].copy(), X_holdout.iloc[idx_B].copy()
    y_A, y_B = y_holdout.iloc[idx_A].copy(), y_holdout.iloc[idx_B].copy()

    # Storage for aggregated results
    all_predictions = {}
    all_y_true = {}
    all_stage_accuracies = []

    # Run 1: A=validation, B=test
    print("\n" + "="*60)
    print("RUN 1: Using Split A for validation, Split B for testing")
    print("="*60)

    preds_run1, accs_run1 = train_and_evaluate_single_run(X_A, X_B, y_A, y_B, "Run 1")
    for i, idx in enumerate(idx_B):
        all_predictions[idx] = preds_run1[i]
        all_y_true[idx] = y_B.iloc[i]
    all_stage_accuracies.append(accs_run1)

    # Run 2: B=validation, A=test (SWAP)
    print("\n" + "="*60)
    print("RUN 2: Using Split B for validation, Split A for testing (SWAP)")
    print("="*60)

    preds_run2, accs_run2 = train_and_evaluate_single_run(X_B, X_A, y_B, y_A, "Run 2")
    for i, idx in enumerate(idx_A):
        all_predictions[idx] = preds_run2[i]
        all_y_true[idx] = y_A.iloc[i]
    all_stage_accuracies.append(accs_run2)

    # Aggregated results
    print("\n" + "="*60)
    print("AGGREGATED RESULTS ACROSS ENTIRE HOLDOUT SET")
    print("="*60)

    sorted_indices = sorted(all_predictions.keys())
    final_preds_all = [all_predictions[i] for i in sorted_indices]
    y_true_all = [all_y_true[i] for i in sorted_indices]

    print("\n== Aggregated Classification Report ==")
    print(classification_report(y_true_all, final_preds_all, labels=ALL_CLASSES,
                               target_names=ALL_CLASSES, digits=4, zero_division=0))

    bal_acc_aggregated = balanced_accuracy_score(y_true_all, final_preds_all)
    print(f"\nAggregated Balanced Accuracy: {bal_acc_aggregated:.4f}")

    cm_aggregated = confusion_matrix(y_true_all, final_preds_all, labels=ALL_CLASSES)
    plot_confusion_matrix_relative(cm_aggregated, class_names=ALL_CLASSES,
                                  title="Aggregated Final Confusion Matrix (Entire Holdout)")

    avg_stage_accs = np.mean(all_stage_accuracies, axis=0)
    print(f"\n== Average Stage Balanced Accuracies ==")
    print(f"Stage 1 (I vs Rest): {avg_stage_accs[0]:.4f}")
    print(f"Stage 2 (DC vs Rest): {avg_stage_accs[1]:.4f}")
    print(f"Stage 3 (Multiclass): {avg_stage_accs[2]:.4f}")
    print(f"Final (Soft-Gated): {avg_stage_accs[3]:.4f}")

    return final_preds_all, [avg_stage_accs[0], avg_stage_accs[1], avg_stage_accs[2], bal_acc_aggregated]

In [ ]:
params1 = {'units': 239, 'learning_rate': 0.0016, 'dropout_rate': 0.364, 'num_hidden_layers': 1, 'batch_size': 110, 'epochs': 46, 'l2_reg': 0.0039, 'activation': "selu", 'optimizer': "sgd"}
#params2 = {'units': 167, 'learning_rate': 0.0054, 'dropout_rate': 0.3488, 'num_hidden_layers': 1, 'batch_size': 21, 'epochs': 50, 'l2_reg': 0.00612, 'activation': 'selu', 'optimizer': 'rmsprop'}
params2 = {'units': 236, 'learning_rate': 0.0092, 'dropout_rate': 0.484, 'num_hidden_layers': 1, 'batch_size': 84, 'epochs': 59, 'l2_reg': 0.0018, 'activation': "selu", 'optimizer': "sgd"}
params3 = {'units': 86, 'learning_rate': 0.01, 'dropout_rate': 0.2, 'num_hidden_layers': 3, 'batch_size': 71, 'epochs': 86, 'l2_reg': 0.000001, 'activation': "selu", 'optimizer': "rmsprop"}
params4 = {'units': 150, 'learning_rate': 0.01, 'dropout_rate': 0.5, 'num_hidden_layers': 1, 'batch_size': 84, 'epochs': 88, 'l2_reg': 0.01, 'activation': "selu", 'optimizer': "sgd"}

train = pd.read_csv('df_cluster.csv')
test = pd.read_csv('df_cluster_VAL.csv')

train = labelize(train)
test = labelize(test)
test.columns = train.columns # some column names have slighlty different names

CORR_THRESHOLD = 0.9

X_train = train.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = train['y']
X_test = test.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_test = test['y']

accs = []
for i in range(10):
    report, b_acc = run_cascade_TEST(X_train, y_train, X_test, y_test)
    accs.append(b_acc)
for element in accs:
    element = [float(a) for a in element]
    print(element)

In [ ]:
def run_multi_TEST(X_train, y_train, X_holdout, y_holdout, params):

    def train_and_evaluate_single_run(X_val, X_test, y_val, y_test, run_name):

        # Encode labels
        le = LabelEncoder()
        y_train_enc = le.fit_transform(y_train)
        y_val_enc = le.transform(y_val)
        y_test_enc = le.transform(y_test)
        class_names = le.classes_

        # Feature filter
        ff = FeatureFilter(corr_threshold=CORR_THRESHOLD)
        ff.fit(X_train)
        X_tr_ff, X_val_ff, X_te_ff = [ff.transform(X) for X in [X_train, X_val, X_test]]

        # Preprocessing
        pre_multi = build_preprocessor_multi(X_tr_ff)
        X_tr_prep, X_val_prep, X_te_prep = [pre_multi.transform(X) for X in [X_tr_ff, X_val_ff, X_te_ff]]
        input_dim = X_tr_prep.shape[1]

        # Build and train model
        gen = BalancedBatchGenerator(X_tr_prep, y_train_enc, batch_size=params["batch_size"])
        model = create_model(input_dim, 4, params, is_binary=False)
        history = model.fit(
            gen,
            validation_data=(X_val_prep, y_val_enc),
            epochs=int(params["epochs"]),
            callbacks=callbacks,
            verbose=0
        )

        # Predictions
        y_pred_prob = model.predict(X_te_prep, verbose=1)
        y_pred_idx = np.argmax(y_pred_prob, axis=1)
        y_test_labels = le.inverse_transform(y_test_enc)
        y_pred_labels = le.inverse_transform(y_pred_idx)

        # Reports
        print(f"\n== Test Set Evaluation - {run_name} ==")
        print(classification_report(y_test_labels, y_pred_labels, labels=class_names, digits=4, zero_division=0))
        bal_acc = balanced_accuracy_score(y_test_labels, y_pred_labels)
        print("Balanced Accuracy:", bal_acc)
        cm = confusion_matrix(y_test_labels, y_pred_labels, labels=class_names)
        print("Confusion Matrix:\n", cm)
        plot_confusion_matrix_relative(cm, class_names=class_names, title=f"Test Confusion Matrix - {run_name}")

        return y_test_labels, y_pred_labels, bal_acc, class_names, history

    # Split holdout set
    sks = StratifiedKFold(n_splits=2, shuffle=True, random_state=RANDOM_STATE)
    idx_A, idx_B = next(sks.split(X_holdout, y_holdout))
    X_A, X_B = X_holdout.iloc[idx_A].copy(), X_holdout.iloc[idx_B].copy()
    y_A, y_B = y_holdout.iloc[idx_A].copy(), y_holdout.iloc[idx_B].copy()

    # Storage for aggregated results
    all_predictions = {}
    all_y_true = {}
    all_balanced_accs = []

    # Run 1: A=validation, B=test
    print("\n" + "="*60)
    print("RUN 1: Using Split A for validation, Split B for testing")
    print("="*60)

    y_test_1, y_pred_1, bal_acc_1, class_names, history = train_and_evaluate_single_run(X_A, X_B, y_A, y_B, "Run 1")
    for i, idx in enumerate(idx_B):
        all_predictions[idx] = y_pred_1[i]
        all_y_true[idx] = y_test_1[i]
    all_balanced_accs.append(bal_acc_1)

    # Run 2: B=validation, A=test (SWAP)
    print("\n" + "="*60)
    print("RUN 2: Using Split B for validation, Split A for testing (SWAP)")
    print("="*60)

    y_test_2, y_pred_2, bal_acc_2, _, history = train_and_evaluate_single_run(X_B, X_A, y_B, y_A, "Run 2")
    for i, idx in enumerate(idx_A):
        all_predictions[idx] = y_pred_2[i]
        all_y_true[idx] = y_test_2[i]
    all_balanced_accs.append(bal_acc_2)

    # Aggregated results
    print("\n" + "="*60)
    print("AGGREGATED RESULTS ACROSS ENTIRE HOLDOUT SET")
    print("="*60)

    sorted_indices = sorted(all_predictions.keys())
    final_preds_all = [all_predictions[i] for i in sorted_indices]
    y_true_all = [all_y_true[i] for i in sorted_indices]

    print("\n== Aggregated Classification Report ==")
    print(classification_report(y_true_all, final_preds_all, labels=class_names, digits=4, zero_division=0))

    bal_acc_aggregated = balanced_accuracy_score(y_true_all, final_preds_all)
    print(f"\nAggregated Balanced Accuracy: {bal_acc_aggregated:.4f}")

    cm_aggregated = confusion_matrix(y_true_all, final_preds_all, labels=class_names)
    print("Aggregated Confusion Matrix:\n", cm_aggregated)
    plot_confusion_matrix_relative(cm_aggregated, class_names=class_names,
                                   title="Aggregated Confusion Matrix (Entire Holdout)")

    avg_bal_acc = np.mean(all_balanced_accs)
    print(f"\n== Average Balanced Accuracy Across Runs ==")
    print(f"Run 1: {all_balanced_accs[0]:.4f}")
    print(f"Run 2: {all_balanced_accs[1]:.4f}")
    print(f"Average: {avg_bal_acc:.4f}")

    return (
        {"y_true": pd.Series(y_true_all).reset_index(drop=True),
         "y_pred": pd.Series(final_preds_all).reset_index(drop=True)},
        bal_acc_aggregated,
        history
    )

In [ ]:
params4 = {'units': 150, 'learning_rate': 0.01, 'dropout_rate': 0.5, 'num_hidden_layers': 1, 'batch_size': 84, 'epochs': 88, 'l2_reg': 0.01, 'activation': "selu", 'optimizer': "sgd"}

train = pd.read_csv('df_cluster.csv')
test = pd.read_csv('df_cluster_VAL.csv')
train = labelize(train)
test = labelize(test)
test.columns = train.columns # some column names have slighlty different names

X_train = train.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = train['y']
X_holdout = test.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_holdout =  test['y']

accs = []
for i in range(10):
    report, b_acc, history = run_multi_TEST(X_train, y_train, X_holdout, y_holdout, params4)
    accs.append(b_acc)
accs = [float(a) for a in accs]
print(accs)
print(statistics.mean(accs), statistics.pstdev(accs) / math.sqrt(len(accs)))

# Testing (not aggregated)

Before continuing, make sure the optimized preprocessing pipeline and the *create_model* function from the "Optimized" section are loaded.

In [ ]:
def run_cascade_TEST(X_train, y_train, X_holdout, y_holdout):
    """
    Train cascade on a training set (with 20% reserved for validation),
    and evaluate on the full holdout set.
    """
    accuracies = []

    # --- Split training set into train (80%) and validation (20%)
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
    train_idx, val_idx = next(sss.split(X_train, y_train))

    X_tr_raw, X_val_raw = X_train.iloc[train_idx].copy(), X_train.iloc[val_idx].copy()
    y_tr, y_val = y_train.iloc[train_idx].copy(), y_train.iloc[val_idx].copy()

    # Use entire holdout set for testing
    X_test_raw = X_holdout.copy()
    y_test = y_holdout.copy()

    # ---- Feature filter
    ff = FeatureFilter(corr_threshold=CORR_THRESHOLD)
    ff.fit(X_tr_raw)
    X_tr_ff = ff.transform(X_tr_raw)
    X_val_ff = ff.transform(X_val_raw)
    X_te_ff = ff.transform(X_test_raw)

    # ---- Preprocessing
    pre1 = build_preprocessor(X_tr_ff, 1)
    X1_tr, X1_val, X1_te = (
        pre1.transform(X_tr_ff),
        pre1.transform(X_val_ff),
        pre1.transform(X_te_ff),
    )

    pre2 = build_preprocessor(X_tr_ff, 2)
    X2_tr, X2_val, X2_te = (
        pre2.transform(X_tr_ff),
        pre2.transform(X_val_ff),
        pre2.transform(X_te_ff),
    )

    pre3 = build_preprocessor(X_tr_ff, 3)
    X3_tr, X3_val, X3_te = (
        pre3.transform(X_tr_ff),
        pre3.transform(X_val_ff),
        pre3.transform(X_te_ff),
    )

    in_dim = X1_tr.shape[1]

    # ========= Stage 1: I vs Rest =========
    print("\n-- Stage 1 (I vs Rest) --")
    y1_tr = build_stage1_labels(y_tr)
    y1_val   = build_stage1_labels(y_val)
    y1_te  = build_stage1_labels(y_test)

    cw1 = compute_class_weight(class_weight="balanced", classes=np.unique(y1_tr), y=y1_tr)
    cw1_dict = {int(cls): float(w) for cls, w in zip(np.unique(y1_tr), cw1)}

    m1 = create_model(in_dim, 1, params1, is_binary=True)
    history1 = m1.fit(X1_tr, y1_tr.values,
           validation_data=(X1_val, y1_val.values),
           epochs=params1["epochs"],
           batch_size=params1["batch_size"],
           callbacks=callbacks,
           verbose=0,
           class_weight=cw1_dict)

    plot_history(history1)

    # --- Threshold tuning on validation
    p1_val = m1.predict(X1_val, verbose=0).ravel()
    thresh1 = find_optimal_threshold(y1_val, p1_val)
    #thresh1 = 0.55 This is the average threshold obtained from the 5 best performing runs
    print(f"Optimal threshold (Stage 1): {thresh1:.3f}")

    # --- Apply tuned threshold on test set
    p1_te = m1.predict(X1_te, verbose=0).ravel()
    pred1_te = (p1_te >= thresh1).astype(int)
    print(classification_report(y1_te, pred1_te, digits=4, zero_division=0))
    print("Balanced Accuracy:", balanced_accuracy_score(y1_te, pred1_te))
    cm1 = confusion_matrix(y1_te, pred1_te)
    plot_confusion_matrix_relative(cm1, class_names=["Rest", "I"], title="Stage 1 (I vs Rest)")
    accuracies.append(balanced_accuracy_score(y1_te, pred1_te))

    # ========= Stage 2: DC vs Rest =========
    print("\n-- Stage 2 (DC vs Rest) --")
    mask2_tr, y2_tr = build_stage2_labels(y_tr)
    mask2_val, y2_val = build_stage2_labels(y_val)
    mask2_te, y2_te = build_stage2_labels(y_test)

    X2_tr_m, X2_val_m, X2_te_m = X2_tr[mask2_tr.values], X2_val[mask2_val.values], X2_te[mask2_te.values]

    cw2 = compute_class_weight(class_weight="balanced", classes=np.unique(y2_tr), y=y2_tr)
    cw2_dict = {int(cls): float(w) for cls, w in zip(np.unique(y2_tr), cw2)}

    m2 = create_model(in_dim, 2, params2, is_binary=True)
    history2 = m2.fit(X2_tr_m, y2_tr,
           validation_data=(X2_val_m, y2_val.values),
           epochs=params2['epochs'],
           batch_size=params2["batch_size"],
           callbacks=callbacks,
           verbose=0,
           class_weight=cw2_dict)

    plot_history(history2)

    # --- Threshold tuning on validation
    p2_val = m2.predict(X2_val_m, verbose=0).ravel()
    thresh2 = find_optimal_threshold(y2_val, p2_val)
    if thresh2 < 0.2:
        thresh2 = 0.2
    print(f"Optimal threshold (Stage 2): {thresh2:.3f}")

    # --- Apply tuned threshold on test set
    p2_te = m2.predict(X2_te_m, verbose=0).ravel()
    pred2_te = (p2_te >= thresh2).astype(int)
    print(classification_report(y2_te, pred2_te, digits=4, zero_division=0))
    print("Balanced Accuracy:", balanced_accuracy_score(y2_te, pred2_te))
    cm2 = confusion_matrix(y2_te, pred2_te)
    plot_confusion_matrix_relative(cm2, class_names=["PID", "DC"], title="Stage 2 (DC vs Rest)")
    accuracies.append(balanced_accuracy_score(y2_te, pred2_te))

    # ========= Stage 3: Multiclass =========
    print("\n-- Stage 3 (Multiclass) --")
    mask3_tr, y3_tr = build_stage3_labels(y_tr)
    mask3_val, y3_val = build_stage3_labels(y_val)
    mask3_te, y3_te = build_stage3_labels(y_test)

    X3_tr_m, X3_val_m, X3_te_m = X3_tr[mask3_tr.values], X3_val[mask3_val.values], X3_te[mask3_te.values]

    #sm = SMOTE(random_state=17, k_neighbors=5)
    #X3_tr_res, y3_tr_res = sm.fit_resample(X3_tr_m, y3_tr.values)

    num_classes = 4
    cw3 = compute_class_weight(class_weight="balanced", classes=np.unique(y3_tr), y=y3_tr)
    cw3_dict = {int(cls): float(w) for cls, w in zip(np.unique(y3_tr), cw3)}

    m3 = create_model(in_dim, 3, params3, is_binary=False)
    history3 = m3.fit(X3_tr_m, y3_tr,
           validation_data=(X3_val_m, y3_val.values),
           epochs=params3["epochs"],
           batch_size=params3["batch_size"],
           callbacks=callbacks,
           verbose=0,
           class_weight=cw3_dict)

    plot_history(history3)

    p3_te = m3.predict(X3_te_m, verbose=0)
    pred3_te_idx = np.argmax(p3_te, axis=1)
    print(classification_report(y3_te, pred3_te_idx, digits=4, zero_division=0))
    print("Balanced Accuracy:", balanced_accuracy_score(y3_te, pred3_te_idx))
    cm3 = confusion_matrix(y3_te, pred3_te_idx)
    plot_confusion_matrix_relative(cm3, class_names=STAGE3_CLASSES, title="Stage 3 (Multiclass)")
    accuracies.append(balanced_accuracy_score(y3_te, pred3_te_idx))

    # ========= Overall predictions (test set only) =========
    pI_all  = m1.predict(X1_te, verbose=0).ravel()
    pDC_all = m2.predict(X2_te, verbose=0).ravel()
    pS3_all = m3.predict(X3_te, verbose=0)

    final_probs = soft_gated_combine_probs(pI_all, pDC_all, pS3_all, thresh1, thresh2)
    final_idx = np.argmax(final_probs, axis=1)
    final_preds = [ALL_CLASSES[i] for i in final_idx]

    print("\n== Soft-Gated Overall (Test Set) ==")
    print(classification_report(y_test, final_preds, labels=ALL_CLASSES,
                                target_names=ALL_CLASSES, digits=4, zero_division=0))
    print("Balanced Accuracy:", balanced_accuracy_score(y_test, final_preds))
    cm_final = confusion_matrix(y_test, final_preds, labels=ALL_CLASSES)
    plot_confusion_matrix_relative(cm_final, class_names=ALL_CLASSES, title="Final Soft-Gated Confusion Matrix")
    accuracies.append(balanced_accuracy_score(y_test, final_preds))

    return final_preds, accuracies

In [ ]:
params1 = {'units': 239, 'learning_rate': 0.0016, 'dropout_rate': 0.364, 'num_hidden_layers': 1, 'batch_size': 110, 'epochs': 46, 'l2_reg': 0.0039, 'activation': "selu", 'optimizer': "sgd"}
#params2 = {'units': 167, 'learning_rate': 0.0054, 'dropout_rate': 0.3488, 'num_hidden_layers': 1, 'batch_size': 21, 'epochs': 50, 'l2_reg': 0.00612, 'activation': 'selu', 'optimizer': 'rmsprop'}
params2 = {'units': 236, 'learning_rate': 0.0092, 'dropout_rate': 0.484, 'num_hidden_layers': 1, 'batch_size': 84, 'epochs': 59, 'l2_reg': 0.0018, 'activation': "selu", 'optimizer': "sgd"}
params3 = {'units': 86, 'learning_rate': 0.01, 'dropout_rate': 0.2, 'num_hidden_layers': 3, 'batch_size': 71, 'epochs': 86, 'l2_reg': 0.000001, 'activation': "selu", 'optimizer': "rmsprop"}
params4 = {'units': 150, 'learning_rate': 0.01, 'dropout_rate': 0.5, 'num_hidden_layers': 1, 'batch_size': 84, 'epochs': 88, 'l2_reg': 0.01, 'activation': "selu", 'optimizer': "sgd"}

train = pd.read_csv('df_cluster.csv')
test = pd.read_csv('df_cluster_VAL.csv')

train = labelize(train)
test = labelize(test)
test.columns = train.columns # some column names have slighlty different names

CORR_THRESHOLD = 0.9

X_train = train.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = train['y']
X_test = test.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_test = test['y']

accs = []
for i in range(10):
    report, b_acc = run_cascade_TEST(X_train, y_train, X_test, y_test)
    accs.append(b_acc)
for element in accs:
    element = [float(a) for a in element]
    print(element)

In [ ]:
def run_multi_TEST(X_train, y_train, X_holdout, y_holdout):
    """
    Train a multiclass model on a training set (with stratified 80/20 split for monitoring),
    and evaluate once on the provided holdout set.
    """

    # Encode labels
    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train)
    y_holdout_enc = le.transform(y_holdout)
    class_names = le.classes_

    # ---- Split training into train/val (80/20 stratified)
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
    tr_idx, val_idx = next(sss.split(X_train, y_train_enc))

    X_tr_raw, X_val_raw = X_train.iloc[tr_idx].copy(), X_train.iloc[val_idx].copy()
    y_tr, y_val = y_train_enc[tr_idx], y_train_enc[val_idx]

    # ---- Feature filter
    ff = FeatureFilter(corr_threshold=CORR_THRESHOLD)
    ff.fit(X_tr_raw)
    X_tr_ff, X_val_ff, X_te_ff = ff.transform(X_tr_raw), ff.transform(X_val_raw), ff.transform(X_holdout)

    # ---- Preprocessing
    pre4 = build_preprocessor(X_tr_ff, 4)
    X_tr_prep, X_val_prep, X_te_prep = (
        pre4.transform(X_tr_ff),
        pre4.transform(X_val_ff),
        pre4.transform(X_te_ff),
    )
    input_dim = X_tr_prep.shape[1]

    # ---- Class weights
    cw = compute_class_weight(class_weight="balanced", classes=np.unique(y_tr), y=y_tr)
    cw_dict = {cls: w for cls, w in zip(np.unique(y_tr), cw)}

    # ---- Build and train model
    model = create_model(input_dim, 4, params4, is_binary=False)
    history = model.fit(
        X_tr_prep, y_tr,
        validation_data=(X_val_prep, y_val),
        epochs=int(params4["epochs"]),
        batch_size=int(params4["batch_size"]),
        callbacks=callbacks,
        verbose=0,
        class_weight=cw_dict
        )

    # ---- Predictions on holdout set
    y_pred_prob = model.predict(X_te_prep, verbose=1)
    y_pred_idx = np.argmax(y_pred_prob, axis=1)

    # Decode back to string labels
    y_ho_labels = le.inverse_transform(y_holdout_enc)
    y_pred_labels = le.inverse_transform(y_pred_idx)

    # ---- Reports
    print("\n== Holdout Evaluation ==")
    print(classification_report(y_ho_labels, y_pred_labels, labels=class_names, digits=4, zero_division=0))
    print("Balanced Accuracy:", balanced_accuracy_score(y_ho_labels, y_pred_labels))
    cm = confusion_matrix(y_ho_labels, y_pred_labels, labels=class_names)
    print("Confusion Matrix:\n", cm)
    plot_confusion_matrix_relative(cm, class_names=class_names, title="Holdout Confusion Matrix")

    return {"y_true": pd.Series(y_ho_labels).reset_index(drop=True),
            "y_pred": pd.Series(y_pred_labels).reset_index(drop=True)}, balanced_accuracy_score(y_ho_labels, y_pred_labels), history


In [ ]:
#params1 = {'units': 239, 'learning_rate': 0.0016, 'dropout_rate': 0.364, 'num_hidden_layers': 1, 'batch_size': 110, 'epochs': 46, 'l2_reg': 0.0039, 'activation': "selu", 'optimizer': "sgd"}
#params2 = {'units': 236, 'learning_rate': 0.0092, 'dropout_rate': 0.484, 'num_hidden_layers': 1, 'batch_size': 84, 'epochs': 59, 'l2_reg': 0.0018, 'activation': "selu", 'optimizer': "sgd"}
#params3 = {'units': 86, 'learning_rate': 0.01, 'dropout_rate': 0, 'num_hidden_layers': 3, 'batch_size': 71, 'epochs': 86, 'l2_reg': 0.000001, 'activation': "selu", 'optimizer': "rmsprop"}
params4 = {'units': 150, 'learning_rate': 0.01, 'dropout_rate': 0.5, 'num_hidden_layers': 1, 'batch_size': 84, 'epochs': 88, 'l2_reg': 0.01, 'activation': "selu", 'optimizer': "sgd"}


#params1 = {'units': 239, 'learning_rate': 0.0016, 'dropout_rate': 0.364, 'num_hidden_layers': 1, 'batch_size': 110, 'epochs': 46, 'l2_reg': 0.0039, 'activation': "selu", 'optimizer': "sgd"}
#params2 = {'units': 236, 'learning_rate': 0.0092, 'dropout_rate': 0.484, 'num_hidden_layers': 1, 'batch_size': 84, 'epochs': 59, 'l2_reg': 0.0018, 'activation': "selu", 'optimizer': "sgd"}
#params3 = {'units': 86, 'learning_rate': 0.01, 'dropout_rate': 0, 'num_hidden_layers': 3, 'batch_size': 71, 'epochs': 86, 'l2_reg': 0.000001, 'activation': "selu", 'optimizer': "rmsprop"}
#params4 = {'units': 150, 'learning_rate': 0.01, 'dropout_rate': 0.5, 'num_hidden_layers': 1, 'batch_size': 84, 'epochs': 88, 'l2_reg': 0.01, 'activation': "selu", 'optimizer': "sgd"}

train = pd.read_csv('df_cluster.csv')
test = pd.read_csv('df_cluster_VAL.csv')

train = labelize(train)
test = labelize(test)
test.columns = train.columns # some column names have slighlty different names

CORR_THRESHOLD = 0.9

X_train = train.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_train = train['y']
X_test = test.drop(columns=['IUIS', 'IUIS extended', 'PCODE','y'])
y_test = test['y']

accs = []
for i in range(5):
    report, b_acc, history = run_multi_TEST(X_train, y_train, X_test, y_test)
    accs.append(float(b_acc))

print(accs)